<a href="https://colab.research.google.com/github/fabiomatricardi/How-I-Built-a-Chatbot-that-Crushed-ChatGPT/blob/main/Only_CPU_MEDIUM_RAG_openVsChatGPT_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **S**ort **o**f **M**ixture **o**f **E**xperts


### Project Orchestartion Outline

Use various LLM stack as part of Document analysis


*   LaMiniFlan-T5-77M for **Summarization**
*   BAAI/bge-base-en-v1.5 Embeddings (only for English) max lenght 512 tokens
*   StableLM-Zephyr-3B GGUF quantized for **QnA** and complex Reasoning

Main points: StableLM only on CPU has quite big inference time. LaMini-783M takes around 2 minutes to generate the questions. LaMini 77M create a Summary in 9 seconds.

I used the reranking function to inject in position -2 the Summary as part of the Context

### Install dependencies

In [ ]:
%%capture
!pip install transformers -U --no-cache-dir
!pip install llama-cpp-python==0.2.34
!pip install langchain
!pip install rich
!pip install tiktoken
!pip install chromadb
!pip install sentence-transformers
!huggingface-cli download TheBloke/stablelm-zephyr-3b-GGUF stablelm-zephyr-3b.Q5_K_S.gguf --local-dir . --local-dir-use-symlinks False
!wget https://github.com/fabiomatricardi/How-I-Built-a-Chatbot-that-Crushed-ChatGPT/raw/main/Article-edited.txt
!wget https://github.com/fabiomatricardi/How-I-Built-a-Chatbot-that-Crushed-ChatGPT/raw/main/Article-original.txt

<img src="https://github.com/fabiomatricardi/How-I-Built-a-Chatbot-that-Crushed-ChatGPT/raw/main/restartRuntime.png" width=800>

In [ ]:
# @title Main imports and models load
from tqdm.rich import trange, tqdm
from rich import console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.text import Text
import warnings
warnings.filterwarnings(action='ignore')
import datetime
from rich.console import Console
console = Console(width=110)
from transformers import pipeline
import os
## Logger file
tstamp = datetime.datetime.now()
tstamp = str(tstamp).replace(' ','_')
logfile = f'{tstamp}_log.txt'
def writehistory(text):
    with open(logfile, 'a', encoding='utf-8') as f:
        f.write(text)
        f.write('\n')
    f.close()

## Load  MBZUAI/LaMini-Flan-T5-77M for summarization
with console.status("Loading ✅ LaMini77M...",spinner="dots12"):
    model77 = pipeline('text2text-generation',model="MBZUAI/LaMini-Flan-T5-77M")
writehistory(f"{str(datetime.datetime.now())} Loaded 🦙 MBZUAI/LaMini-Flan-T5-77M for summarization")
## Load a llama-cpp-python quantized model
from llama_cpp import Llama
with console.status("Loading ✅✅✅✅ stablelm-zephyr-3b with LLAMA.CPP...",spinner="dots12"):
  llm = Llama(
    model_path="/content/stablelm-zephyr-3b.Q5_K_S.gguf",  # Download the model file first
    n_ctx=4096,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=2,            # The number of CPU threads to use, tailor to your system and the resulting performance
  )
writehistory(f"{str(datetime.datetime.now())} Loaded 🧠 stablelm-zephyr-3b.Q5_K_S.gguf for heavy QnA")
# Load Embeddings for MultiLanguage
from langchain.embeddings import HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')
writehistory(f"{str(datetime.datetime.now())} Loaded 🧞‍♂️ BAAI/bge-base-en-v1.5 Embeddings")

#Load text files
with open("/content/Article-original.txt") as f:
  originaltext = f.read()
f.close()
with open("/content/Article-edited.txt") as f:
  editedtext = f.read()
f.close()
writehistory(f"{str(datetime.datetime.now())} [red bold]Original scraped text saved into [bright_green on black]originaltext[/bright_green on black] variable")
writehistory(f"{str(datetime.datetime.now())} [blue bold]Edited scraped text saved into [bright_yellow on black]editedtext[/bright_yellow on black] variable")
writehistory(f"{str(datetime.datetime.now())} Original text lenght: [bright_green on black]{len(originaltext)}")
writehistory(f"{str(datetime.datetime.now())} Edited text lenght: [bright_yellow on black]{len(editedtext)}")

### FlanT5 and StableLM-Zephy QnA functions
- qgen77  --> `qg77 = qgen77(splitted_text_qg, model77,'🎴🎴LaMiniFlanT5-77M')`
- summary77 --> `sum77 = summary77(splitted_text_qg, model77,'🎴LaMiniFlanT5-77M') `
- stableQA --> `q,o = stableQA("What is Science?",250,llm)`
- stableQnA --> `q1,o1 = stableQnA("What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital Markets Act?",editedtext,450,llm)`

for summarization and question generation we can use 2 different splits
- for QnA and embedding retrieval a smaller chunk too
- all of them in tokens count

In [ ]:
# @title
# Different chunks for the functions (Summary/Question Generation)
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
TOKENtext_splitter = TokenTextSplitter(chunk_size=430, chunk_overlap=20)
splitted_text_sum = TOKENtext_splitter.split_text(editedtext) #create a list
TOKENtext_splitter = TokenTextSplitter(chunk_size=280, chunk_overlap=20)
splitted_text_qg = TOKENtext_splitter.split_text(editedtext) #create a list
TOKENtext_splitter = TokenTextSplitter(chunk_size=150, chunk_overlap=20)
splitted_text_qna = TOKENtext_splitter.split_text(editedtext) #create a list


#Function for T5 model to summarize text based on textsplitted chunks base
def summary77(textplitted, model, modelname):
  writehistory(f"{str(datetime.datetime.now())} Using {modelname} for summarization task...")
  start = datetime.datetime.now()
  with console.status("Generating summary...",spinner="dots12"):
    summary ="SUMMARY:\n"
    for item in textplitted:
      text = item
      template_summary = f'''Text: {text}

Write a complete summary of the above text.
'''
      res = model(template_summary, temperature=0.3, repetition_penalty=1.3, max_length=300, do_sample=True)[0]['generated_text']
      #console.print(res)
      #console.print('---')
      summary = summary + res + '\n'
  delta = datetime.datetime.now() - start
  writehistory(f"{str(datetime.datetime.now())} [bold]----------------------------")
  writehistory(f"{str(datetime.datetime.now())} {summary}")
  writehistory(f"{str(datetime.datetime.now())} [red1 bold]Full SUMMARY Completed in {delta}")
  return summary

#Function for T5 model to generate 2 questions for each chunk of text based on textsplitted chunks base
def qgen77(textplitted, model, modelname):
  writehistory(f"{str(datetime.datetime.now())} Using {modelname} for for Question generation task -  2 PER CHUNK...")
  start = datetime.datetime.now()
  quest2 = []
  with console.status("Generating Questions...",spinner="dots12"):
    for item in textplitted:
      text = item
      template_qg = f'''{text}\n\n
write two important questions about the above text.
Questions:
1.
2.
'''
      res = model(template_qg, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
      ed_res = res.replace('? ','?#')
      list_res = ed_res.split('#')
      for i in list_res:
        #a = i[3:]  REMOVED TRUNK on the list, generated by LaMini
        quest2.append(i[3:])

  writehistory(f"{str(datetime.datetime.now())} [bold]----------------------------")
  writehistory(f"{str(datetime.datetime.now())} {quest2}")
  return quest2




#Function for general inference of the model, no context required
def stableQA(question,maxtokens,model):
  """
  basic generation with StableLM / any llama.cpp loaded model
  question -> string
  maxtokens -> int, number of max tokens to generate
  model -> llama-cpp-python instance // here is StableLM-Zephyr-3B
  RETURNS question, ouotput -> str
  """
  import datetime
  start = datetime.datetime.now()
  prompt = question
  template = f"<|user|>\n{prompt}<|endoftext|>\n<|assistant|>"
  with console.status("StableLM-Zephyr-3B AI is working ✅✅✅ ...",spinner="dots12"):
    output = model(
      template, # Prompt
      temperature=0.3,
      max_tokens=maxtokens,  # Generate up to 512 tokens
      stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
      echo=False        # Whether to echo the prompt
    )
    delta = datetime.datetime.now() - start
    console.print(f"Question: [bright_green on black]{prompt}")
    console.print(output['choices'][0]['text'])
    console.print(f"Completed in: [bold red]{delta}")
    return question, output['choices'][0]['text']

#Function for QnA over a Context - the context is pure string text
def stableQnA(question,contesto,maxtokens,model):
  """
  basic generation with StableLM / any llama.cpp loaded model
  question -> string
  contesto -> string, parsed page_content from document objects
  maxtokens -> int, number of max tokens to generate
  model -> llama-cpp-python instance // here is StableLM-Zephyr-3B
  RETURNS question, ouotput -> str
  """
  context = contesto
  query = question
  import datetime
  start = datetime.datetime.now()
  template = f"""<|user|>\nGiven this text extracts:\n-----\n{context}\n-----\nPlease answer the question. Your answer must be informative and organized into bullet points. If the question is unanswerable, ""say \"unanswerable\".\nQuestion: {query}<|endoftext|>\n<|assistant|>"""
  with console.status("StableLM-Zephyr-3B AI is working ✅✅✅ ...",spinner="dots12"):
    output = model(
      template, # Prompt
      temperature=0.3,
      max_tokens=maxtokens,  # Generate up to 512 tokens
      stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
      echo=False        # Whether to echo the prompt
    )
    delta = datetime.datetime.now() - start
    console.print(f"[bright_green bold on black]Question: {query}")
    console.print(output['choices'][0]['text'])
    console.print(f"Completed in: [bold red]{delta}")
    return question, output['choices'][0]['text']

In [ ]:
sum77 = summary77(splitted_text_qg, model77,'🎴LaMiniFlanT5-77M')
console.print(sum77)

Output()

In [ ]:
qg77 = qgen77(splitted_text_qg, model77,'🎴🎴LaMiniFlanT5-77M')
console.print(Markdown(f"## Suggested Questions"))
for items in qg77:
  console.print(Markdown(f"- {items}"))

Output()

In [ ]:
console.print(Markdown(f"# Reply to the Question without Langchain"))
q1,o1 = stableQnA("What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital Markets Act?",editedtext,450,llm)

Output()

Question: What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital
Markets Act?

- Apple will allow alternative app stores and other new options for app developers in order to comply with 
Europe's Digital Markets Act (DMA), which goes into effect on March 7.
- The company stated that its compliance adheres to the letter of the law, while also emphasizing its efforts 
to ensure iOS devices remain secure and malware-free.
- Apple is introducing new fee structures for developers who take advantage of the new freedoms: a "core 
technology fee" of €.50 per year for apps with more than 1 million downloads, a reduced App Store commission 
(from 30% to 17%), and discounted rates for certain developers.
- The iOS changes will include notarization for iOS apps, authorization for marketplace developers, and 
disclosures on alternative payments.
- Users in the EU will see a browser choice screen in iOS 17.4 to pick a default browser.
- It is up to the European Commission to determine whether Apple's plan fully complies with the DMA, and 
whether these changes boost competition and improve experiences for users.

Completed in: 0:06:23.966433

### Create a VectorDB with HugginFace embeddings and Reeanking QnA with StalbLM-Zephyr

- createDB --> `db3 = createDB(docdocs, hf_embeddings, 'db-128-0-BGEBase')`
- QnA_Rerank_Stable --> `r1,t1,q1,rd1 = QnA_Rerank_Stable(db3,3, docsum, question,llm)`

In [ ]:
# @title
# Splitting into 128 tokens chunks
TOKENtext_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=0)
splitted_text_qna = TOKENtext_splitter.split_text(editedtext) #create a list

def createDB(docs, embeddings, dbname):
  """
  Function that create a Chroma Vector store of splitted documents
  with provided embeddings, and save it locally.
  docs: text_splitter docs object
  embeddings: HuggingFace embeddings object
  dbname: string, the name of the persistent db
  RETURN  the Chroma db object
  """
  from langchain.vectorstores import Chroma
  import datetime
  start = datetime.datetime.now()
  db = Chroma.from_documents(docs, embeddings,persist_directory=f"./{dbname}")
  stop = datetime.datetime.now()
  delta = stop-start
  writehistory(f"{str(datetime.datetime.now())} Vector db generated in {delta}")
  return db

#Function for QnA from ChramDB with Reranking and Summary Injection
def QnA_Rerank_Stable(db,k, summarization, query,model):
  """
  return the generated answer to a similarity search query
  with Re-ranking of k elements
  and run the QnA chain with Prompt Template
  inputs:
  db -> ChromaDB object instance
  k -> number of hits for the similarity search (be aware of Max Context Lenght!)
       must be >= 3
  summarization -> Langchain Document object with the Summarization
  query -> string witht he question for the similarity search
  llm -> llama-cpp-python model instance
  return res: str generated answer by llm
         delta: time object, duration of the llm generation
         ques: the question string
         reordered_docs: list of LangChainDocuments ReRanked with Summarization
  """
  import datetime
  start = datetime.datetime.now()
  # Create a retriever
  retriever = db.as_retriever(search_kwargs={"k": k})
  from langchain.document_transformers import LongContextReorder
  from langchain.chains import StuffDocumentsChain, LLMChain
  from langchain.prompts import PromptTemplate
  # Get relevant documents ordered by relevance score
  context_set = retriever.get_relevant_documents(query)
  #print(str(context_set))
  # Reorder the documents:
  # Less relevant document will be at the middle of the list and more
  # relevant elements at beginning / end.
  reordering = LongContextReorder()
  reordered_docs = reordering.transform_documents(context_set)
  reordered_docs.insert(-1,summarization)
  #print(str(reordered_docs))
  # We prepare and run a custom Stuff chain with reordered docs as context.
  # Override prompts
  context = ''
  for i in reordered_docs:
    context += i.page_content
  template = f"""<|user|>\nGiven this text extracts:\n-----\n{context}\n-----\nPlease answer the question. Your answer must be precise, informative and organized into bullet points. If the question is unanswerable, ""say \"unanswerable\".\nQuestion: {query}<|endoftext|>\n<|assistant|>"""
  with console.status("StableLM-Zephyr-3B AI is working ✅✅✅ ...",spinner="dots12"):
    output = model(
      template, # Prompt
      temperature=0.3,
      max_tokens=450,  # Generate up to 512 tokens
      stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
      echo=False        # Whether to echo the prompt
    )
  result = output['choices'][0]['text']
  delta = datetime.datetime.now() - start
  return result, delta, query, reordered_docs

def evidenzia_res(keys, fulltext,summary):
  from rich.text import Text
  mytest = fulltext + summary.page_content
  text = Text(mytest)
  for item in keys:
    l = len(item.page_content)
    x = mytest.find(item.page_content)
    text.stylize("black on bright_yellow", x, (x+l))
  console.print(text)
  return text

#Create 2 Langchain documents: for the splitted text and for the summary
from langchain.schema.document import Document
docdocs = []
for i in range(0,len(splitted_text_qna)):
  docdocs.append(Document(page_content = splitted_text_qna[i],
                          metadata = {'source': '/content/2024-01-27 11.23.05 Apple s iOS App Store announces_edited.txt',
                              'title': "Apple's iOS App Store announces sweeping changes in the EU",
                              'author': 'Ashley Gold, author of Axios Pro',
                              'url' : 'https://www.axios.com/2024/01/25/apple-app-store-eu-changes',
                              }))
docsum = Document(page_content = sum77, metadata = {
                  'source': '/content/2024-01-27 11.23.05 Apple s iOS App Store announces_edited.txt',
                  'title': "Apple's iOS App Store announces sweeping changes in the EU",
                  'author': 'Ashley Gold, author of Axios Pro',
                  'url' : 'https://www.axios.com/2024/01/25/apple-app-store-eu-changes',
                  'type': 'summary'})

In [ ]:
db3 = createDB(docdocs, hf_embeddings, 'db-128-0-BGEBase')

In [ ]:
from rich.markdown import Markdown
console.print(Markdown("## QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary"))
question = "What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital Markets Act?"
r1,t1,q1,rd1 = QnA_Rerank_Stable(db3,3, docsum, question,llm)
console.print(f"[bold red1]Question: {q1}")
console.print(r1)
console.print("---")
console.print(f"generated in {t1}")

Llama.generate: prefix-match hit

Output()

Question: What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital
Markets Act?

- Apple will allow alternative app stores and other new options for app developers in order to comply with 
Europe's Digital Markets Act (DMA).
- The company is tightening its strict rules, including security and privacy concerns, as well as providing 
services that are best-in-class and free of spam or harmful content.
- Apple has implemented new safeguards for iOS devices, such as "each first annual install per year" for 
developers who have more than 1 million downloads, and a fee structure applying to apps that switch to their 
own marketplaces or remain on the App Store.
- The European Commission will determine whether Apple's plan fully complies with the DMA.
- Apple is introducing new browser choice screens in iOS 17.4 to pick a default browser.
- There are no plans to expand these options outside of Europe, as Apple cites risk.

---

generated in 0:04:42.131637

In [ ]:
from rich.markdown import Markdown
console.print(Markdown("# QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary"))
question = "What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital Markets Act?"
r1,t1,q1,rd1 = QnA_Rerank_Stable(db3,5, docsum, question,llm)
console.print(f"[bold red1]Question: {q1}")
console.print(r1)
console.print("---")
console.print(f"generated in {t1}")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                     QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Llama.generate: prefix-match hit

Output()

Question: What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital
Markets Act?

- Apple will allow alternative app stores and other new options for app developers in order to comply with 
Europe's Digital Markets Act (DMA).
- The company will introduce notarization for iOS apps, authorization for marketplace developers, and 
disclosures on alternative payments.
- Approximate 99% of developers will pay the same or less in fees.
- Apple says it has been in touch with developers about the coming changes and that its compliance adheres to 
the letter of the law.
- The European Commission will determine whether Apple's plan fully complies with the DMA.

---

generated in 0:05:04.067964

QnA without a Vector dB

In [ ]:
q1,o1 = stableQnA("What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital Markets Act?",editedtext,450,llm)

Llama.generate: prefix-match hit

Output()

Question: What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital
Markets Act?

- Apple will allow alternative app stores and other new options for app developers in order to comply with 
Europe's Digital Markets Act (DMA).
- The DMA requires Apple to loosen its strict rules requiring developers to rely on the App Store for 
distribution and payment processing.
- Apple is making these changes while ensuring iOS devices remain secure and malware-free, but it has no plans
to expand these options outside of Europe due to risk.
- New fee structures have been introduced for developers who take advantage of the new freedoms: a "core 
technology fee" will be charged for apps with more than 1 million downloads per year.
- The App Store commission will drop from 30% to 17%, and the discounted rate for certain developers will 
decrease from 15% to 10%.
- No commission will be charged to developers who want to distribute and facilitate in-app payments outside of
the App Store.
- Users in the EU will see a browser choice screen in iOS 17.4 to pick a default browser.
- It's up to the European Commission to determine whether Apple's plan fully complies with the DMA, and 
whether these changes boost competition and improve experiences for users.

Completed in: 0:05:56.704152

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

---
---

---

---






---



---


---



# Further studies

### Full functions for QnA and Reranking with summary

```
# load from disk
db3 = Chroma(persist_directory="./chroma_db480tok-20", embedding_function=hf_embeddings)
```



In [ ]:
#from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
#llm = HuggingFacePipeline(pipeline=model783)

In [ ]:
def QnA_Rerank_Stable(db,k, summarization, query,model):
  """
  return the generated answer to a similarity search query
  with Re-ranking of k elements
  and run the QnA chain with Prompt Template
  inputs:
  db -> ChromaDB object instance
  k -> number of hits for the similarity search (be aware of Max Context Lenght!)
       must be >= 3
  summarization -> Langchain Document object with the Summarization
  query -> string witht he question for the similarity search
  llm -> llama-cpp-python model instance
  return res: str generated answer by llm
         delta: time object, duration of the llm generation
         ques: the question string
         reordered_docs: list of LangChainDocuments ReRanked with Summarization
  """
  import datetime
  start = datetime.datetime.now()
  # Create a retriever
  retriever = db.as_retriever(search_kwargs={"k": k})
  from langchain.document_transformers import LongContextReorder
  from langchain.chains import StuffDocumentsChain, LLMChain
  from langchain.prompts import PromptTemplate
  # Get relevant documents ordered by relevance score
  context_set = retriever.get_relevant_documents(query)
  #print(str(context_set))
  # Reorder the documents:
  # Less relevant document will be at the middle of the list and more
  # relevant elements at beginning / end.
  reordering = LongContextReorder()
  reordered_docs = reordering.transform_documents(context_set)
  reordered_docs.insert(-1,summarization)
  #print(str(reordered_docs))
  # We prepare and run a custom Stuff chain with reordered docs as context.
  # Override prompts
  context = ''
  for i in reordered_docs:
    context += i.page_content
  template = f"""<|user|>\nGiven this text extracts:\n-----\n{context}\n-----\nPlease answer the question. Your answer must be precise, informative and organized into bullet points. If the question is unanswerable, ""say \"unanswerable\".\nQuestion: {query}<|endoftext|>\n<|assistant|>"""
  with console.status("StableLM-Zephyr-3B AI is working ✅✅✅ ...",spinner="dots12"):
    output = model(
      template, # Prompt
      temperature=0.3,
      max_tokens=450,  # Generate up to 512 tokens
      stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
      echo=False        # Whether to echo the prompt
    )
  result = output['choices'][0]['text']
  delta = datetime.datetime.now() - start
  return result, delta, query, reordered_docs

def evidenzia_res(keys, fulltext,summary):
  from rich.text import Text
  mytest = fulltext + summary.page_content
  text = Text(mytest)
  for item in keys:
    l = len(item.page_content)
    x = mytest.find(item.page_content)
    text.stylize("black on bright_yellow", x, (x+l))
  console.print(text)
  return text

## QnA_Rerank_Plus with 🧠💎 StablLM-Zephyr-3B using Reranking and Summary

In [ ]:
from rich.markdown import Markdown
console.print(Markdown("## QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary"))
question = "What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital Markets Act?"
r1,t1,q1,rd1 = QnA_Rerank_Stable(db3,3, docsum, question,llm)
console.print(f"[bold red1]Question: {q1}")
console.print(r1)
console.print("---")
console.print(f"generated in {t1}")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                     QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Llama.generate: prefix-match hit

Output()

Question: What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital
Markets Act?

- Apple will allow alternative app stores and other new options for app developers to comply with Europe's 
Digital Markets Act (DMA).
- The company claims that its services are best-in-class and ensure smooth user experiences free of spam or 
harmful content.
- Users in the EU will see a browser choice screen in iOS 17.4 to pick a default browser.
- Apple is introducing new fees for developers who take advantage of these free freedoms, with approximately 
99% of developers paying the same or less fees for their iOS changes.
- Epic Games CEO Tim Sweeney has criticized Apple's plan as an instance of Malicious Compliance and has 
rejected the company's use of notarization and malware scanning processes to undermine competition.
- The European Commission will determine whether Apple's plan fully complies with the DMA.

---

generated in 0:04:18.568234

In [ ]:
from rich.markdown import Markdown
console.print(Markdown("# QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary"))
question = "What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital Markets Act?"
r1,t1,q1,rd1 = QnA_Rerank_Stable(db3,5, docsum, question,llm)
console.print(f"[bold red1]Question: {q1}")
console.print(r1)
console.print("---")
console.print(f"generated in {t1}")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                     QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Llama.generate: prefix-match hit

Output()

Question: What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital
Markets Act?

- Apple will allow alternative app stores and other new options for app developers to comply with Europe's 
Digital Markets Act.
- The company will introduce notarization for iOS apps, authorization for marketplace developers, and 
disclosures on alternative payments.
- Users in the EU will see a browser choice screen in iOS 17.4 to pick a default browser.
- Apple claims that approximately 99% of developers will pay the same or less fees for these changes.
- The European Commission has announced that there will be no commission charged to developers who want to 
distribute and facilitate in-app payments outside of the App Store.
- Epic Games CEO Tim Sweeney has criticized Apple's plan as Malicious Compliance, arguing that they are trying
to undermine competition and impose Apple taxes on transactions not involved in the process.

---

generated in 0:07:13.929793

In [ ]:
from rich.markdown import Markdown
console.print(Markdown("# QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary"))
question = "What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital Markets Act?"
r1,t1,q1,rd1 = QnA_Rerank_Stable(db3,9, docsum, question,llm)
console.print(f"[bold red1]Question: {q1}")
console.print(r1)
console.print("---")
console.print(f"generated in {t1}")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                     QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Llama.generate: prefix-match hit

Output()

Question: What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital
Markets Act?

- Apple will allow alternative app stores and other new options for app developers to comply with Europe's 
Digital Markets Act (DMA).
- The company will introduce notarization for iOS apps, authorization for marketplace developers, and 
disclosures on alternative payments.
- Users in the EU will see a browser choice screen in iOS 17.4 to pick a default browser.
- Apple claims that approximately 99% of developers will pay the same or less fees for their iOS changes, 
including the new freedoms.
- The App Store commission, for developers who continue to use it, will drop from 30% to 17%.
- A "core technology fee" of €.50 will be charged for "each first annual install per year" for apps with more 
than 1 million downloads, to be counted once per Apple account.

---

generated in 0:06:32.678458

## Further studies section

### Load a Llama.cpp model details

In [ ]:
!huggingface-cli download TheBloke/stablelm-zephyr-3b-GGUF stablelm-zephyr-3b.Q5_K_S.gguf --local-dir . --local-dir-use-symlinks False

In [ ]:
!wget https://huggingface.co/TheBloke/stablelm-zephyr-3b-GGUF/resolve/main/stablelm-zephyr-3b.Q5_K_S.gguf?download=true



```
# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
  model_path="/content/stablelm-zephyr-3b.Q5_K_S.gguf",  # Download the model file first
  n_ctx=4096,  # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
)

# Simple inference example
output = llm(
  "<|user|>\n{prompt}<|endoftext|>\n<|assistant|>", # Prompt
  max_tokens=512,  # Generate up to 512 tokens
  stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)
```



"<|user|>\n{prompt}<|endoftext|>\n<|assistant|>"

In [ ]:
#from langchain.callbacks.manager import CallbackManager
#from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
#from langchain_community.llms import LlamaCpp
from llama_cpp import Llama

llm = Llama(
  model_path="/content/stablelm-zephyr-3b.Q5_K_S.gguf",  # Download the model file first
  n_ctx=4096,  # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=2,            # The number of CPU threads to use, tailor to your system and the resulting performance
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'general.file_type': '16', 'tokenizer.chat_template': "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}", 'tokenizer.ggml.unknown_token_id': '0', 'general.architecture': 'stablelm', 'general.name': 'source', 'stablelm.embedding_length': '2560', 'stablelm.context_length': '4096', 'stablelm.block_count': '32', 'stablelm.feed_forward_length': '6912', 'stablelm.use_parallel_residual': 'true', 'tokenize

In [ ]:
# Different chunks for the functions (Summary/Question Generation)
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
TOKENtext_splitter = TokenTextSplitter(chunk_size=430, chunk_overlap=20)
splitted_text_sum = TOKENtext_splitter.split_text(editedtext) #create a list
TOKENtext_splitter = TokenTextSplitter(chunk_size=280, chunk_overlap=20)
splitted_text_qg = TOKENtext_splitter.split_text(editedtext) #create a list
TOKENtext_splitter = TokenTextSplitter(chunk_size=150, chunk_overlap=20)
splitted_text_qna = TOKENtext_splitter.split_text(editedtext) #create a list


#Function for T5 model to summarize text based on textsplitted chunks base
def summary77(textplitted, model, modelname):
  writehistory(f"{str(datetime.datetime.now())} Using {modelname} for summarization task...")
  start = datetime.datetime.now()
  with console.status("Generating summary...",spinner="dots12"):
    summary ="SUMMARY:\n"
    for item in textplitted:
      text = item
      template_summary = f'''Text: {text}

Write a complete summary of the above text.
'''
      res = model(template_summary, temperature=0.3, repetition_penalty=1.3, max_length=300, do_sample=True)[0]['generated_text']
      #console.print(res)
      #console.print('---')
      summary = summary + res + '\n'
  delta = datetime.datetime.now() - start
  writehistory(f"{str(datetime.datetime.now())} [bold]----------------------------")
  writehistory(f"{str(datetime.datetime.now())} {summary}")
  writehistory(f"{str(datetime.datetime.now())} [red1 bold]Full SUMMARY Completed in {delta}")
  return summary

#Function for T5 model to generate 2 questions for each chunk of text based on textsplitted chunks base
def qgen77(textplitted, model, modelname):
  writehistory(f"{str(datetime.datetime.now())} Using {modelname} for for Question generation task -  2 PER CHUNK...")
  start = datetime.datetime.now()
  quest2 = []
  with console.status("Generating Questions...",spinner="dots12"):
    for item in textplitted:
      text = item
      template_qg = f'''{text}\n\n
write two important questions about the above text.
Questions:
1.
2.
'''
      res = model(template_qg, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
      ed_res = res.replace('? ','?#')
      list_res = ed_res.split('#')
      for i in list_res:
        #a = i[3:]  REMOVED TRUNK on the list, generated by LaMini
        quest2.append(i[3:])

  writehistory(f"{str(datetime.datetime.now())} [bold]----------------------------")
  writehistory(f"{str(datetime.datetime.now())} {quest2}")
  return quest2




#Function for general inference of the model, no context required
def stableQA(question,maxtokens,model):
  """
  basic generation with StableLM / any llama.cpp loaded model
  question -> string
  maxtokens -> int, number of max tokens to generate
  model -> llama-cpp-python instance // here is StableLM-Zephyr-3B
  RETURNS question, ouotput -> str
  """
  import datetime
  start = datetime.datetime.now()
  prompt = question
  template = f"<|user|>\n{prompt}<|endoftext|>\n<|assistant|>"
  with console.status("StableLM-Zephyr-3B AI is working ✅✅✅ ...",spinner="dots12"):
    output = model(
      template, # Prompt
      temperature=0.3,
      max_tokens=maxtokens,  # Generate up to 512 tokens
      stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
      echo=False        # Whether to echo the prompt
    )
    delta = datetime.datetime.now() - start
    console.print(f"Question: [bright_green on black]{prompt}")
    console.print(output['choices'][0]['text'])
    console.print(f"Completed in: [bold red]{delta}")
    return question, output['choices'][0]['text']

#Function for QnA over a Context - the context is pure string text
def stableQnA(question,contesto,maxtokens,model):
  """
  basic generation with StableLM / any llama.cpp loaded model
  question -> string
  contesto -> string, parsed page_content from document objects
  maxtokens -> int, number of max tokens to generate
  model -> llama-cpp-python instance // here is StableLM-Zephyr-3B
  RETURNS question, ouotput -> str
  """
  context = contesto
  query = question
  import datetime
  start = datetime.datetime.now()
  template = f"""<|user|>\nGiven this text extracts:\n-----\n{context}\n-----\nPlease answer the question. Your answer must be informative and organized into bullet points. If the question is unanswerable, ""say \"unanswerable\".\nQuestion: {query}<|endoftext|>\n<|assistant|>"""
  with console.status("StableLM-Zephyr-3B AI is working ✅✅✅ ...",spinner="dots12"):
    output = model(
      template, # Prompt
      temperature=0.3,
      max_tokens=maxtokens,  # Generate up to 512 tokens
      stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
      echo=False        # Whether to echo the prompt
    )
    delta = datetime.datetime.now() - start
    console.print(f"[bright_green bold on black]Question: {query}")
    console.print(output['choices'][0]['text'])
    console.print(f"Completed in: [bold red]{delta}")
    return question, output['choices'][0]['text']

- Usage example

In [ ]:
q,o = stableQA("What is Science?",250,llm)

Llama.generate: prefix-match hit

Output()

Question: What is Science?

Science is a systematic way of studying the natural world and gaining new knowledge and understanding through 
observation, experiment, analysis, and theory. Amsler-key elements of science include:

1. Observation: The process of gathering information about natural phenomena around us.
2. Experimentation: The systematic testing of hypotheses by manipulating variables to observe how they affect 
the subject being studied.
3. Analysis: The interpretation and organization of data collected through observation and experimentation.
4. Theory: A well-substantiated explanation or model that predicts natural phenomena and can be tested and 
refined through experimentation and observation.
5. Experimentation: The systematic testing of hypotheses by manipulating variables to observe how they affect 
the subject being studied.
6. Replicability: The ability to reproduce results obtained by others, ensuring validity and reliability of 
scientific findings.
7. Objectivity: The pursuit of knowledge without bias or preconceptions about what is true or false.
8. Scientific Method: A step-by-step process used in the scientific community to guide research and 
investigation, including hypothesis formulation, experimentation, analysis, theory testing, and peer review.

In [ ]:
q1,o1 = stableQnA("What are the changes made by Apple to its iOS App Store in order to comply with Europe's Digital Markets Act?",editedtext,450,llm)

Llama.generate: prefix-match hit

Output()

Question: What is Science?

Apple has made several changes to its iOS App Store in order to comply with Europe's Digital Markets Act 
(DMA), which goes into effect on March 7. These changes include allowing alternative app stores and other new 
options for app developers, introducing new fee structures for developers who take advantage of these 
freedoms, and including notarization for iOS apps, authorization for marketplace developers, and disclosures 
on alternative payments. The App Store commission will also be reduced from 30% to 17%, and developers who 
want to distribute and facilitate in-app payments outside of the App Store won't be charged a fee. Users in 
the EU will see a browser choice screen in iOS 17.4 to pick a default browser. Apple claims that these changes
are being made with the aim of maintaining iOS devices' security and malware-free status while reducing risks 
posed by the DMA, although the company has no plans to expand these options outside of Europe due to risk.

Completed in: 0:06:31.239495

In [ ]:
q1,o1 = stableQnA("What are the changes made by Apple to its iOS App Store in order to comply with Europe's Digital Markets Act?",editedtext,450,llm)

Llama.generate: prefix-match hit

Output()

Question: What are the changes made by Apple to its iOS App Store in order to comply with Europe's Digital 
Markets Act?

- Apple will allow alternative app stores and other new options for app developers as part of its efforts to 
comply with Europe's Digital Markets Act (DMA), which goes into effect on March 7.
- The changes made by Apple include introducing new fee structures, notarization for iOS apps, authorization 
for marketplace developers, and disclosures on alternative payments.
- A "core technology fee" will be charged to developers whose apps have more than 1 million downloads per 
year, with a fee of €.50 per first annual install. This applies to both apps that switch to their own 
marketplaces or remain on the App Store.
- The App Store commission for developers who continue to use it will drop from 30% to 17%. Discounted rates 
for certain developers will also be reduced from 15% to 10%.
- No commission will be charged to developers who want to distribute and facilitate in-app payments outside of
the App Store, with Apple stating that approximately 99% of developers will pay the same or less in fees.
- Users in the EU will see a browser choice screen in iOS 17.4 to pick a default browser.

Completed in: 0:02:31.605118

In [ ]:
# "1. What are the iOS changes that will be implemented to ensure compliance with Europe's Digital Markets Act law?"
q1,o1 = stableQnA("1. What are the iOS changes that will be implemented to ensure compliance with Europe's Digital Markets Act law?",editedtext,450,llm)

Llama.generate: prefix-match hit

Output()

Question: 1. What are the iOS changes that will be implemented to ensure compliance with Europe's Digital 
Markets Act law?

1. Apple will allow alternative app stores and other new options for app developers in order to comply with 
Europe's Digital Markets Act (DMA). 
   - This is a change from Apple's strict rules requiring developers to rely on the App Store for distribution
and payment processing. 

2. The iOS changes will include notarization for iOS apps, authorization for marketplace developers, and 
disclosures on alternative payments. 
   - Notarization helps ensure app safety by verifying their authenticity. 
   - Authorization allows developers who use alternative marketplaces to distribute their apps without going 
through the App Store. 
   - Disclosures on alternative payments help inform users about payment methods outside of the App Store.

3. Apple is introducing new fee structures for developers who take advantage of the new freedoms. 
   - A "core technology fee" will be charged to developers whose apps have more than 1 million downloads, with
a rate of €.50 per first annual install per year. 
   - The App Store commission will drop from 30% to 17%, and the discounted rates for certain developers will 
change as well.

4. Users in the EU will see a browser choice screen in iOS 17.4 to pick a default browser. 

5. Apple's compliance adheres to the letter of the law, but the company has no plans to expand these options 
outside of Europe due to security risks.

6. The European Commission will determine whether Apple fully complies with the DMA.

Completed in: 0:03:04.007134

- for summarization and question generation we can use 2 different splits
- for QnA and embedding retrieval a smaller chunk too
- all of them in tokens count

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
TOKENtext_splitter = TokenTextSplitter(chunk_size=430, chunk_overlap=20)
splitted_text_sum = TOKENtext_splitter.split_text(editedtext) #create a list
TOKENtext_splitter = TokenTextSplitter(chunk_size=280, chunk_overlap=20)
splitted_text_qg = TOKENtext_splitter.split_text(editedtext) #create a list
TOKENtext_splitter = TokenTextSplitter(chunk_size=150, chunk_overlap=20)
splitted_text_qna = TOKENtext_splitter.split_text(editedtext) #create a list

In [ ]:
# @title
def summary(textplitted, model, modelname):
  console.print(f'Using {modelname} for summarization task...')
  start = datetime.datetime.now()
  with console.status("Generating summary...",spinner="dots12"):
    summary =""
    for item in textplitted:
      text = item
      template_summary = f'''ARTICLE: {text}

      What is a one-paragraph summary of the above article?

      '''
      res = model(template_summary, temperature=0.3, repetition_penalty=1.3, max_length=300, do_sample=True)[0]['generated_text']
      #console.print(res)
      #console.print('---')
      summary = summary + res + '\n'
  delta = datetime.datetime.now() - start
  console.print('[bold]----------------------------')
  console.print(summary)
  console.print(f"[red1 bold]Full SUMMARY Completed in {delta}")
  return summary

sum783 = summary(splitted_text_sum, model783,'🎴🎴🎴LaMiniFlanT5-783M')

In [ ]:
def summary77(textplitted, model, modelname):
  writehistory(f"{str(datetime.datetime.now())} Using {modelname} for summarization task...")
  start = datetime.datetime.now()
  with console.status("Generating summary...",spinner="dots12"):
    summary ="SUMMARY:\n"
    for item in textplitted:
      text = item
      template_summary = f'''Text: {text}

Write a complete summary of the above text.
'''
      res = model(template_summary, temperature=0.3, repetition_penalty=1.3, max_length=300, do_sample=True)[0]['generated_text']
      #console.print(res)
      #console.print('---')
      summary = summary + res + '\n'
  delta = datetime.datetime.now() - start
  writehistory(f"{str(datetime.datetime.now())} [bold]----------------------------")
  writehistory(f"{str(datetime.datetime.now())} {summary}")
  writehistory(f"{str(datetime.datetime.now())} [red1 bold]Full SUMMARY Completed in {delta}")
  return summary

In [ ]:
# @title
sum77 = summary77(splitted_text_sum, model77,'🎴LaMiniFlanT5-77M')

Using 🎴LaMiniFlanT5-77M for summarization task...

Output()

----------------------------

SUMMARY:
Apple's iOS App Store is allowing alternative app stores and other new options for app developers to comply 
with Europe's Digital Markets Act, which requires the company to loosen its strict rules. The changes are 
expected to take place in Europe post-March 7, but it has no plans to expand these options outside of Europe 
due to risk. It also introduces new safeguards that reduce vulnerabilities, but does not eliminate harmful 
content.
The European Commission is deciding whether Apple's plan fullycomplies with the Digital Markets Act law, which
has been introduced by developers in Europe. It will be applied to apps that switch to their own marketplaces 
or remain on the App Store and include notarization for iOS apps, authorization for marketplace developers, 
disclosures on alternative payments, and browser choice screen in iOS 17.4. However, it is up to the EU to 
determine whether Apple plans fully comply with the DMA.

Full SUMMARY Completed in 0:00:09.089113

In [ ]:
sum77 = summary77(splitted_text_qg, model77,'🎴LaMiniFlanT5-77M')

Using 🎴LaMiniFlanT5-77M for summarization task...

Output()

----------------------------

SUMMARY:
Apple's iOS App Store will allow alternative app stores and other new options for app developers to comply 
with Europe's Digital Markets Act, which is set to go into effect on March 7. The company has resisted any 
changes due to security and privacy. It also claims that its services are best-in-class and ensure smooth user
experiences free of spam or harmful content. Spotify previewed the latest version in Europe post-March 7 this 
week, and it says it has been in touch with developers about the coming changes and compliance adheres to the 
letter of the law.
Apple has implemented strict policies for developers to ensure iOS devices remain secure and malware-free, but
it is not yet imposing any plans outside of Europe. The company believes that the changes will be necessary in
order to protect users' data from potential threats. It also introduces new fees for developers who take 
advantage of these free freedoms.
The European Commission has announced that there will be no commission charged to developers who want to 
distribute and facilitate in-app payments outside of the App Store. Apple said that approximately 99% of 
developers will pay the same or less fees for their iOS changes, including notarization for iOS apps, 
authorization for marketplace developers, and disclosures on alternative payment options. Users in the EU will
see a browser choice screen in iOS 17.4 to pick a default browser. Epic Games CEO Tim Sweeney stated that this
is an example of Malicious Compliance, but it's up to the European Commission to determine whether Apple's 
plan fully comply with the DMA.

Full SUMMARY Completed in 0:00:14.793238



---



---



In [ ]:
def qgen77(textplitted, model, modelname):
  writehistory(f"{str(datetime.datetime.now())} Using {modelname} for for Question generation task -  2 PER CHUNK...")
  start = datetime.datetime.now()
  quest2 = []
  with console.status("Generating Questions...",spinner="dots12"):
    for item in textplitted:
      text = item
      template_qg = f'''{text}\n\n
write two important questions about the above text.
Questions:
1.
2.
'''
      res = model(template_qg, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
      ed_res = res.replace('? ','?#')
      list_res = ed_res.split('#')
      for i in list_res:
        #a = i[3:]  REMOVED TRUNK on the list, generated by LaMini
        quest2.append(i[3:])

  writehistory(f"{str(datetime.datetime.now())} [bold]----------------------------")
  writehistory(f"{str(datetime.datetime.now())} {quest2}")
  return quest2

In [ ]:
qg77 = qgen77(splitted_text_qg, model77,'🎴🎴LaMiniFlanT5-77M')

Using 🎴🎴LaMiniFlanT5-77M for for Question generation task -  2 PER CHUNK...

Output()

----------------------------

[
    'What changes will Apple make to its iOS App Store in the EU?',
    "How does it affect the company's compliance with Europe's Digital Markets Act?",
    "What is the reason behind Apple's changes to iOS 17.4?",
    'How does it impact its policies on developers and how do they respond to these changes?',
    "What is the latest update to Apple's iOS changes?",
    'How does it impact its decision-making process in Europe and how do they affect users?'
]



---



---



---



### not used

In [ ]:
# @title
def qgen(textplitted, model, modelname):
  console.print(f'Using {modelname} for for Question generation task -  2 PER CHUNK...')
  start = datetime.datetime.now()
  quest2 = []
  with console.status("Generating Questions...",spinner="dots12"):
    for item in textplitted:
      text = item
      template_qg = f'''{text}.\nAsk two relevant question about this text.
      '''
      res = model(template_qg, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
      ed_res = res.replace('? ','?#')
      list_res = ed_res.split('#')
      for i in list_res:
        #a = i[3:]  REMOVED TRUNK on the list, generated by LaMini
        quest2.append(i)

  console.print('[bold]----------------------------')
  console.print(quest2)
  return quest2

qg783 = qgen(splitted_text_qg, model783,'🎴🎴🎴LaMiniFlanT5-783M')

In [ ]:
for question in qg77:
  context = editedtext
  query = question
  q1,o1 = stableQnA(query,context,450,llm)
  console.print('[bold]----------------------------')

Output()

Question: What changes are Apple making to its iOS App Store and how does it affect the EU's Digital Markets 
Act?

- Apple will allow alternative app stores and other new options for app developers in order to comply with 
Europe's Digital Markets Act (DMA), which goes into effect on March 7.
- The company stated that its compliance adheres to the letter of the law, while also emphasizing that it has 
no plans to expand these options outside of Europe due to security concerns.
- Apple is introducing new fee structures for developers who take advantage of the new freedoms: a "core 
technology fee" of €.50 per year for apps with more than 1 million downloads, and reduced commission rates for
certain developers.
- The iOS App Store will include notarization for iOS apps, authorization for marketplace developers, and 
disclosures on alternative payments as part of these changes.
- Users in the EU will see a browser choice screen in iOS 17.4 to pick a default browser.
- Apple believes that after the DMA-imposed changes go through, people will start to understand why Apple has 
maintained its strict policies for developers.

Completed in: 0:06:49.375010

----------------------------

...

Output()

Question: How does it impact the competition and user experience for iPhone app developers?

Impact on competition and user experience for iPhone app developers:

- Apple announces sweeping changes in response to Europe's Digital Markets Act, allowing alternative app 
stores and other new options for app developers.
- The company believes that after the DMA-imposed changes go through, people will start to understand why 
Apple has maintained its strict policies for developers.
- New fee structures have been introduced: a "core technology fee" of €.50 per year for apps with more than 1 
million downloads; reduced commission rates (from 30% to 17%) for developers who continue to use the App 
Store, and no commission charged for those distributing and facilitating in-app payments outside of it.
- Notarization for iOS apps, authorization for marketplace developers, and disclosures on alternative payments
will be included as part of these changes.
- Users in the EU will see a browser choice screen in iOS 17.4 to pick a default browser.
- Apple's compliance adheres to the letter of the law but the company has no plans to expand these options 
outside of Europe due to risk concerns.
- The impact on competition and user experience remains to be seen as it is up to the European Commission to 
determine whether Apple's plan fully complies with the DMA, and iPhone app developers face a new world of 
choices.

Completed in: 0:02:39.044530

----------------------------



---



---



---



In [ ]:
def qnAgen(textplitted, summary, quest2, model, modelname):
  console.print(f"[bold blue]Generating replies with {modelname}...")
  console.print('[bold]----------------------------')
  i = 0
  for items in textplitted:
    context = items + summary
    question = quest2[i]
    template_qna = f'''Read this and answer the question. Your answer must be informative. If the question is unanswerable, ""say \"unanswerable\".\n\n{context}\n\n{question}'''
    res = model(template_qna, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
    console.print(f"[bold blue]{question}")
    console.print(res)
    console.print('---')
    i += 1
    question = quest2[i]
    template_qna = f'''Read this and answer the question. Your answer must be informative. If the question is unanswerable, ""say \"unanswerable\".\n\n{context}\n\n{question}'''
    res = model(template_qna, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
    console.print(f"[bold blue]{question}")
    console.print(res)
    console.print('---')
    i += 1


In [ ]:
qnAgen(splitted_text_qg, sum77, qg783, model77,'🎴LaMiniFlanT5-77M')
## got an error since 77M model cannot follow the isntruction to generate 2 questions per chunk

Generating replies with 🎴LaMiniFlanT5-77M...

----------------------------

What is the Digital Markets Act and how does it affect Apple's App Store?

The Digital Markets Act is a legal framework that regulates the use of alternative app stores and other new 
options for app developers in Europe. It affects Apple's App Store by providing flexibility to allow 
alternative apps stores and other new options for app developers, including an "walled garden" system.

---

...

The changes to iOS that will be implemented in response to Europe's new Digital Markets Act law include 
notarization for iOS apps, authorization for marketplace developers and disclosures on alternative payments.

---

Who is Tim Sweeney and what is his opinion on Apple's plan to thwart the DMA?

Tim Sweeney is Epic Games CEO. He believes that Apple's plan to thwart Europe's new Digital Markets Act law is
a devious new instance of Malicious Compliance and strongly rejects it.

---

### Highlither Function


In [ ]:
# key = "notarization for iOS apps"
# mytest = "The iOS changes will include notarization for iOS apps, authorization for marketplace developers and disclosures on alternative payments."
def evidenzia(key, fulltext):
  from rich.text import Text
  mytest = fulltext
  text = Text(mytest)
  l = len(key)
  x = mytest.find(key)
  text.stylize("black on bright_yellow", x, (x+l))
  console.print(text)
  return text

In [ ]:
a = """Apple's sweeping changes to its iOS App Store in Europe have significant implications for smaller companies,
as Spotify recently previewed what it will look like after March 7."""
b = """for smaller companies,
as Spotify"""
s =evidenzia(b,a)

Apple's sweeping changes to its iOS App Store in Europe have significant implications for smaller companies,
as Spotify recently previewed what it will look like after March 7.

### Create a VectorDB with HugginFace embeddings

In [ ]:
TOKENtext_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=0)
splitted_text_qna = TOKENtext_splitter.split_text(editedtext) #create a list

In [ ]:
def createDB(docs, embeddings, dbname):
  """
  Function that create a Chroma Vector store of splitted documents
  with provided embeddings, and save it locally.
  docs: text_splitter docs object
  embeddings: HuggingFace embeddings object
  dbname: string, the name of the persistent db
  RETURN  the Chroma db object
  """
  from langchain.vectorstores import Chroma
  import datetime
  start = datetime.datetime.now()
  db = Chroma.from_documents(docs, embeddings,persist_directory=f"./{dbname}")
  stop = datetime.datetime.now()
  delta = stop-start
  writehistory(f"{str(datetime.datetime.now())} Vector db generated in {delta}")
  return db


In [ ]:
from langchain.schema.document import Document
docdocs = []
for i in range(0,len(splitted_text_qna)):
  docdocs.append(Document(page_content = splitted_text_qna[i],
                          metadata = {'source': '/content/2024-01-27 11.23.05 Apple s iOS App Store announces_edited.txt',
                              'title': "Apple's iOS App Store announces sweeping changes in the EU",
                              'author': 'Ashley Gold, author of Axios Pro',
                              'url' : 'https://www.axios.com/2024/01/25/apple-app-store-eu-changes',
                              }))
docsum = Document(page_content = sum77, metadata = {
                  'source': '/content/2024-01-27 11.23.05 Apple s iOS App Store announces_edited.txt',
                  'title': "Apple's iOS App Store announces sweeping changes in the EU",
                  'author': 'Ashley Gold, author of Axios Pro',
                  'url' : 'https://www.axios.com/2024/01/25/apple-app-store-eu-changes',
                  'type': 'summary'})

In [ ]:
db3 = createDB(docdocs, hf_embeddings, 'db-128-0-BGEBase')

Vector db generated in 0:00:05.348339


```
# load from disk
db3 = Chroma(persist_directory="./chroma_db480tok-20", embedding_function=hf_embeddings)
```



### Full functions for QnA and Reranking with summary

In [ ]:
#from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
#llm = HuggingFacePipeline(pipeline=model783)

In [ ]:
def QnA_Rerank_Stable(db,k, summarization, query,model):
  """
  return the generated answer to a similarity search query
  with Re-ranking of k elements
  and run the QnA chain with Prompt Template
  inputs:
  db -> ChromaDB object instance
  k -> number of hits for the similarity search (be aware of Max Context Lenght!)
       must be >= 3
  summarization -> Langchain Document object with the Summarization
  query -> string witht he question for the similarity search
  llm -> llama-cpp-python model instance
  return res: str generated answer by llm
         delta: time object, duration of the llm generation
         ques: the question string
         reordered_docs: list of LangChainDocuments ReRanked with Summarization
  """
  import datetime
  start = datetime.datetime.now()
  # Create a retriever
  retriever = db.as_retriever(search_kwargs={"k": k})
  from langchain.document_transformers import LongContextReorder
  from langchain.chains import StuffDocumentsChain, LLMChain
  from langchain.prompts import PromptTemplate
  # Get relevant documents ordered by relevance score
  context_set = retriever.get_relevant_documents(query)
  #print(str(context_set))
  # Reorder the documents:
  # Less relevant document will be at the middle of the list and more
  # relevant elements at beginning / end.
  reordering = LongContextReorder()
  reordered_docs = reordering.transform_documents(context_set)
  reordered_docs.insert(-1,summarization)
  #print(str(reordered_docs))
  # We prepare and run a custom Stuff chain with reordered docs as context.
  # Override prompts
  context = ''
  for i in reordered_docs:
    context += i.page_content
  template = f"""<|user|>\nGiven this text extracts:\n-----\n{context}\n-----\nPlease answer the question. Your answer must be precise, informative and organized into bullet points. If the question is unanswerable, ""say \"unanswerable\".\nQuestion: {query}<|endoftext|>\n<|assistant|>"""
  with console.status("StableLM-Zephyr-3B AI is working ✅✅✅ ...",spinner="dots12"):
    output = model(
      template, # Prompt
      temperature=0.3,
      max_tokens=450,  # Generate up to 512 tokens
      stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
      echo=False        # Whether to echo the prompt
    )
  result = output['choices'][0]['text']
  delta = datetime.datetime.now() - start
  return result, delta, query, reordered_docs

def evidenzia_res(keys, fulltext,summary):
  from rich.text import Text
  mytest = fulltext + summary.page_content
  text = Text(mytest)
  for item in keys:
    l = len(item.page_content)
    x = mytest.find(item.page_content)
    text.stylize("black on bright_yellow", x, (x+l))
  console.print(text)
  return text

## QnA_Rerank_Plus with 🧠💎 StablLM-Zephyr-3B using Reranking and Summary

In [ ]:
from rich.markdown import Markdown
console.print(Markdown("# QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary"))
question = "What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital Markets Act?"
r1,t1,q1,rd1 = QnA_Rerank_Stable(db3,3, docsum, question,llm)
console.print(f"[bold red1]Question: {q1}")
console.print(r1)
console.print("---")
console.print(f"generated in {t1}")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                     QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Llama.generate: prefix-match hit

Output()

Question: What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital
Markets Act?

- Apple will allow alternative app stores and other new options for app developers to comply with Europe's 
Digital Markets Act (DMA).
- The company claims that its services are best-in-class and ensure smooth user experiences free of spam or 
harmful content.
- Users in the EU will see a browser choice screen in iOS 17.4 to pick a default browser.
- Apple is introducing new fees for developers who take advantage of these free freedoms, with approximately 
99% of developers paying the same or less fees for their iOS changes.
- Epic Games CEO Tim Sweeney has criticized Apple's plan as an instance of Malicious Compliance and has 
rejected the company's use of notarization and malware scanning processes to undermine competition.
- The European Commission will determine whether Apple's plan fully complies with the DMA.

---

generated in 0:04:18.568234

In [ ]:
from rich.markdown import Markdown
console.print(Markdown("# QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary"))
question = "What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital Markets Act?"
r1,t1,q1,rd1 = QnA_Rerank_Stable(db3,5, docsum, question,llm)
console.print(f"[bold red1]Question: {q1}")
console.print(r1)
console.print("---")
console.print(f"generated in {t1}")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                     QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Llama.generate: prefix-match hit

Output()

Question: What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital
Markets Act?

- Apple will allow alternative app stores and other new options for app developers to comply with Europe's 
Digital Markets Act.
- The company will introduce notarization for iOS apps, authorization for marketplace developers, and 
disclosures on alternative payments.
- Users in the EU will see a browser choice screen in iOS 17.4 to pick a default browser.
- Apple claims that approximately 99% of developers will pay the same or less fees for these changes.
- The European Commission has announced that there will be no commission charged to developers who want to 
distribute and facilitate in-app payments outside of the App Store.
- Epic Games CEO Tim Sweeney has criticized Apple's plan as Malicious Compliance, arguing that they are trying
to undermine competition and impose Apple taxes on transactions not involved in the process.

---

generated in 0:07:13.929793

In [ ]:
from rich.markdown import Markdown
console.print(Markdown("# QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary"))
question = "What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital Markets Act?"
r1,t1,q1,rd1 = QnA_Rerank_Stable(db3,9, docsum, question,llm)
console.print(f"[bold red1]Question: {q1}")
console.print(r1)
console.print("---")
console.print(f"generated in {t1}")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                     QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Llama.generate: prefix-match hit

Output()

Question: What are the major changes Apple is implementing in the EU App Store in response to Europe's Digital
Markets Act?

- Apple will allow alternative app stores and other new options for app developers to comply with Europe's 
Digital Markets Act (DMA).
- The company will introduce notarization for iOS apps, authorization for marketplace developers, and 
disclosures on alternative payments.
- Users in the EU will see a browser choice screen in iOS 17.4 to pick a default browser.
- Apple claims that approximately 99% of developers will pay the same or less fees for their iOS changes, 
including the new freedoms.
- The App Store commission, for developers who continue to use it, will drop from 30% to 17%.
- A "core technology fee" of €.50 will be charged for "each first annual install per year" for apps with more 
than 1 million downloads, to be counted once per Apple account.

---

generated in 0:06:32.678458



---



---



---



In [ ]:
from rich.markdown import Markdown
console.print(Markdown("# QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary"))
for question in qg77:
  r1,t1,q1,rd1 = QnA_Rerank_Stable(db3,2, docsum, question,llm)
  console.print(f"[bold red1]Question: {q1}")
  console.print(r1)
  console.print("---")
  console.print(f"generated in {t1}")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                     QnA GENERATED BY 🧠💎 StablLM-Zephyr-3B with RERANKING and Summary                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Llama.generate: prefix-match hit

Output()

Question: What changes are Apple making to its iOS App Store and how does it affect the EU's Digital Markets 
Act?

Apple is making the following changes to its iOS App Store to comply with Europe's Digital Markets Act (DMA):

1. Allowing alternative app stores and new options for app developers
2. Introducing new safeguards, including encryption and security measures, to reduce risks associated with 
DMA-imposed changes and protect developers from potential threats
3. Implementing new fees for developers who have achieved more than 1 million downloads on their own platforms
or remain on the App Store
4. No commission charged to developers who want to distribute and facilitate in-app payments outside of the 
App Store
5. Inclusion of notarization for iOS apps, authorization for marketplace developers, and disclosures on 
alternative payment options
6. Possible browser choice screen in iOS 17.4 for users in the EU to pick a default browser
7. The company believes that these changes will help maintain its strict policies for developers and ensure 
smooth user experiences free from spam or harmful content
8. Apple claims new safeguards across every change reduce, but do not eliminate, risks posed to EU users by 
the DMA.

...

Output()

Question: How does it impact the competition and user experience for iPhone app developers?

- Apple allows alternative app stores and new options for app developers to comply with Europe's Digital 
Markets Act (DMA)
- New safeguards include encryption, security measures, notarization for iOS apps, authorization for 
marketplace developers, and disclosures on alternative payments
- No commission charged to developers who want to distribute and facilitate in-app payments outside of the App
Store
- Users in EU may see a browser choice screen in iOS 17.4 to pick a default browser
- Impact on competition: iPhone app developers face a whole new world of choices, which may boost competition
- Impact on user experience: Time will tell if the changes improve experiences for users

---

generated in 0:01:30.078728

In [ ]:
def evidenzia_res(keys, fulltext,summary):
  from rich.text import Text
  mytest = fulltext + summary.page_content
  text = Text(mytest)
  for item in keys:
    l = len(item.page_content)
    x = mytest.find(item.page_content)
    text.stylize("black on bright_yellow", x, (x+l))
  console.print(text)
  return text

In [ ]:
a = evidenzia_res(rd1,editedtext,docsum)

Apple's iOS App Store announces sweeping changes in the EU
https://www.axios.com/2024/01/25/apple-app-store-eu-changes
Author: Ashley Gold, author of Axios Pro


Apple will allow alternative app stores and other new options for app developers in order to comply with 
Europe's Digital Markets Act, the company said Thursday.

Driving the news: Europe's major tech competition law, set to go into effect March 7, requires Apple to loosen
its strict rules requiring developers to rely on the App Store for distribution and payment processing.

Apple has long resisted any changes to its App Store rules, citing security and privacy. The company also 
argues its services for developers are best-in-class and ensure smooth user experiences free of spam or 
harmful content.

Why it matters: Apple is being forced to make sweeping changes to one of its flagship "walled garden" systems 
in what some smaller companies are seeing as a major win.

Spotify previewed what its app will look like in Europe post-March 7 this week.
Apple said it has been in touch with developers about the coming changes and that its compliance adheres to 
the letter of the law.

What they're saying: Apple is making these changes with the caveat that the company has done everything it can
to ensure iOS devices remain secure and malware-free, but it has no plans to expand these options outside of 
Europe, citing risk.

The company believes that after the DMA-imposed changes go through, people will start to understand why Apple 
has maintained its strict policies for developers.
"Across every change, Apple is introducing new safeguards that reduce — but don't eliminate — new risks the 
DMA poses to EU users," the company said in a release.

Yes, but: Apple is also introducing new fee structures for developers who take advantage of the new freedoms.

Details: As part of iOS 17.4, Apple will charge a "core technology fee" of €.50 for "each first annual install
per year" for developers whose apps have achieved more than 1 million downloads, to be counted once per Apple 
account.

That fee will apply to apps that switch to their own marketplaces or remain on the App Store. Apple says the 
fee "reflects the many ways Apple creates value for developers' businesses."
The App Store commission, for developers who continue to use it, will drop from 30% to 17%. The discounted 
rate for certain developers will drop from 15% to 10%.
No commission will be charged to developers who want to distribute and facilitate in-app payments outside of 
the App Store. Apple said that approximate 99% of developers will pay the same or less in fees.

Go deeper: The iOS changes will include notarization for iOS apps, authorization for marketplace developers 
and disclosures on alternative payments. Apple says in the release that if apps opt out of Apple's system, the
company "has less ability to address other risks" like "scam, fraud, abuse" or "objectionable or harmful" 
content.

Users in the EU will also see a browser choice screen in iOS 17.4 to pick a default browser.

The other side: Epic Games CEO Tim Sweeney said on X: "Apple's plan to thwart Europe's new Digital Markets Act
law is a devious new instance of Malicious Compliance... Epic has always supported the notion of Apple 
notarization and malware scanning for apps, but we strongly reject Apple's twisting this process to undermine 
competition and continue imposing Apple taxes on transactions they're not involved in."

What to watch: It's up to the European Commission to determine whether Apple's plan fully complies with the 
DMA. iPhone app developers face a whole new world of choices, and time will tell if the changes boost 
competition and improve experiences for users.
SUMMARY:
Apple's iOS App Store will allow alternative app stores and other new options for app developers to comply 
with Europe's Digital Markets Act, which requires developers to rely on the App Store for distribution and 
payment processing. The company has long resi



---



---



---



In [ ]:
r1,t1,q1,rd1 = QnA_Rerank_Plus(db3,2, docsum, qg783[-1][3:],model783)
console.print(f"[bold red1]Question: {q1}")
console.print(r1)
console.print("---")
console.print(f"generated in {t1}")

Question: Who is Tim Sweeney and what is his opinion on Apple's plan to thwart the DMA?

Tim Sweeney is the CEO of Epic Games.

---

generated in 0:00:16.307546

In [ ]:
r1,t1,q1,rd1 = QnA_Rerank_Plus(db3,2, docsum, qg783[-1][3:],model77)
console.print(f"[bold red1]Question: {q1}")
console.print(r1)
console.print("---")
console.print(f"generated in {t1}")

Question: Who is Tim Sweeney and what is his opinion on Apple's plan to thwart the DMA?

Tim Sweeney is Epic Games CEO. His opinion on Apple's plan to thwart the DMA is that it undermines competition
by imposing Apple taxes on transactions they aren't involved in.

---

generated in 0:00:03.384285

## QnA_Rerank_Plus with LaMini Models using Reranking and Summary

In [ ]:
from rich.markdown import Markdown
console.print(Markdown("# QnA GENERATED BY 🦙 **783M LaMini**"))
for question in qg783:
  r1,t1,q1,rd1 = QnA_Rerank_Plus(db3,2, docsum, question,model783)
  console.print(f"[bold red1]Question: {q1}")
  console.print(r1)
  console.print("---")
  console.print(f"generated in {t1}")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                      QnA GENERATED BY 🦙 783M LaMini                                       ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Question: What is the Digital Markets Act and how does it affect Apple's App Store?

The Digital Markets Act is a law in Europe that requires Apple to loosen its strict rules for developers to 
use the App Store for distribution and payment processing. This change has been announced by Apple, but it 
still needs to be implemented before it can become effective.

---

generated in 0:00:22.019572

...

generated in 0:00:24.551566

Question: Who is Tim Sweeney and what is his opinion on Apple's plan to thwart the DMA?

Tim Sweeney is the CEO of Epic Games.

---

generated in 0:00:11.535485

In [ ]:
from rich.markdown import Markdown
console.print(Markdown("# QnA GENERATED BY 🦙 77M LaMini"))
for question in qg783:
  r1,t1,q1,rd1 = QnA_Rerank_Plus(db3,2, docsum, question,model77)
  console.print(f"[bold red1]Question: {q1}")
  console.print(r1)
  console.print("---")
  console.print(f"generated in {t1}")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                       QnA GENERATED BY 🦙 77M LaMini                                       ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Question: What is the Digital Markets Act and how does it affect Apple's App Store?

The Digital Markets Act is a legal framework that requires Apple to regulate the use of alternative app stores
and other new options for developers in order to comply with Europe's digital markets law. It affects Apple's 
App Store by requiring them to loosen its strict rules requiring developers to rely on the App Store for 
distribution and payment processing, which has been made in order to ensure security and privacy of iOS 
devices.

---

generated in 0:00:03.570560

...

generated in 0:00:03.155102

Question: Who is Tim Sweeney and what is his opinion on Apple's plan to thwart the DMA?

Tim Sweeney is Epic Games CEO and his opinion on Apple's plan to thwart the DMA is that it undermines 
competition by imposing Apple taxes on transactions they aren't involved in.

---

generated in 0:00:02.185500

### Huighlight the chunks from the entire text

In [ ]:
def evidenzia_res(keys, fulltext,summary):
  from rich.text import Text
  mytest = fulltext + '\n\nSUMMARY:\n' + summary.page_content
  text = Text(mytest)
  for item in keys:
    l = len(item.page_content)
    x = mytest.find(item.page_content)
    text.stylize("black on bright_yellow", x, (x+l))
  console.print(text)
  return text

In [ ]:
a = evidenzia_res(rd1,editedtext,docsum)

Apple's iOS App Store announces sweeping changes in the EU
https://www.axios.com/2024/01/25/apple-app-store-eu-changes
Author: Ashley Gold, author of Axios Pro


Apple will allow alternative app stores and other new options for app developers in order to comply with 
Europe's Digital Markets Act, the company said Thursday.

Driving the news: Europe's major tech competition law, set to go into effect March 7, requires Apple to loosen
its strict rules requiring developers to rely on the App Store for distribution and payment processing.

Apple has long resisted any changes to its App Store rules, citing security and privacy. The company also 
argues its services for developers are best-in-class and ensure smooth user experiences free of spam or 
harmful content.

Why it matters: Apple is being forced to make sweeping changes to one of its flagship "walled garden" systems 
in what some smaller companies are seeing as a major win.

Spotify previewed what its app will look like in Europe post-March 7 this week.
Apple said it has been in touch with developers about the coming changes and that its compliance adheres to 
the letter of the law.

What they're saying: Apple is making these changes with the caveat that the company has done everything it can
to ensure iOS devices remain secure and malware-free, but it has no plans to expand these options outside of 
Europe, citing risk.

The company believes that after the DMA-imposed changes go through, people will start to understand why Apple 
has maintained its strict policies for developers.
"Across every change, Apple is introducing new safeguards that reduce — but don't eliminate — new risks the 
DMA poses to EU users," the company said in a release.

Yes, but: Apple is also introducing new fee structures for developers who take advantage of the new freedoms.

Details: As part of iOS 17.4, Apple will charge a "core technology fee" of €.50 for "each first annual install
per year" for developers whose apps have achieved more than 1 million downloads, to be counted once per Apple 
account.

That fee will apply to apps that switch to their own marketplaces or remain on the App Store. Apple says the 
fee "reflects the many ways Apple creates value for developers' businesses."
The App Store commission, for developers who continue to use it, will drop from 30% to 17%. The discounted 
rate for certain developers will drop from 15% to 10%.
No commission will be charged to developers who want to distribute and facilitate in-app payments outside of 
the App Store. Apple said that approximate 99% of developers will pay the same or less in fees.

Go deeper: The iOS changes will include notarization for iOS apps, authorization for marketplace developers 
and disclosures on alternative payments. Apple says in the release that if apps opt out of Apple's system, the
company "has less ability to address other risks" like "scam, fraud, abuse" or "objectionable or harmful" 
content.

Users in the EU will also see a browser choice screen in iOS 17.4 to pick a default browser.

The other side: Epic Games CEO Tim Sweeney said on X: "Apple's plan to thwart Europe's new Digital Markets Act
law is a devious new instance of Malicious Compliance... Epic has always supported the notion of Apple 
notarization and malware scanning for apps, but we strongly reject Apple's twisting this process to undermine 
competition and continue imposing Apple taxes on transactions they're not involved in."

What to watch: It's up to the European Commission to determine whether Apple's plan fully complies with the 
DMA. iPhone app developers face a whole new world of choices, and time will tell if the changes boost 
competition and improve experiences for users.


SUMMARY:
Apple has announced that it will allow alternative app stores and other options for developers to comply with 
Europe's Digital Markets Act, which requires the company to loosen its strict rules requiring developers to 
rely on the App Store for distribution 

## Old stuff

### test singolo da me... llmChain pipeline sucks

In [ ]:
context = ''
for i in rd1:
  context += i.page_content
query = qg77[0]
mypronmp = f"""Read this and answer the question. Your answer must be informative. If the question is unanswerable, ""say \"unanswerable\".

{context}

{query}"""

In [ ]:
res = model783(mypronmp, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
console.print(f"[bold red1]Question: {query}")
console.print(res)

Question: What are the changes made by Apple to its iOS App Store in order to comply with Europe's Digital 
Markets Act?

The changes made by Apple to its iOS App Store in order to comply with Europe's Digital Markets Act include a 
fee structure of €.50 per first annual install, an increase in the cost of app development for developers who 
have more than 1 million downloads, and a discount on the App Store commission for those who continue to use 
it.

In [ ]:
context = ''
for i in rd1:
  context += i.page_content

In [ ]:
console.print(rd1)

[
    Document(
        page_content="Apple's iOS App Store announces sweeping changes in the 
EU\nhttps://www.axios.com/2024/01/25/apple-app-store-eu-changes\nAuthor: Ashley Gold, author of\xa0Axios 
Pro\n\n\nApple will allow alternative app stores and other new options for app developers in order to comply 
with Europe's Digital Markets Act, the company said Thursday.\n\nDriving the news: Europe's major tech 
competition law, set to go into effect March 7, requires Apple to loosen its strict rules requiring developers
to rely on the App Store for distribution and payment processing.\n\nApple has long resisted any changes to 
its App Store rules, citing security and privacy. The company also argues its services for developers",
        metadata={
            'author': 'Ashley Gold, author of Axios Pro',
            'source': '/content/2024-01-27 11.23.05 Apple s iOS App Store announces_edited.txt',
            'title': "Apple's iOS App Store announces sweeping changes in the EU",
            'url': 'https://www.axios.com/2024/01/25/apple-app-store-eu-changes'
        }
    ),
    Document(
        page_content="Apple has announced changes to its iOS App Store in order to comply with Europe's 
Digital Markets Act, which requires developers to rely on the app store for distribution and payment 
processing. The company has maintained strict policies regarding security and privacy but will now introduce 
new safeguards that reduce risks, including a fee structure of €.50 per first annual install. Spotify is 
expected to launch an update to its app after March 7, while others are still waiting to see what it will look
like in Europe.\nApple has announced changes to its iOS operating system that will increase the cost of app 
development for developers who have more than 1 million downloads. The core technology fee of €.50 per year 
will be applied to apps that switch to their own marketplaces or remain on the App Store, while the App Store 
commission will drop from 30% to 17%. Developers who continue to use the system will also receive a discounted
rate of 15% to 10%. Additionally, users in Europe will see a browser choice screen where they can choose a 
default browser. However, Epic Games CEO Tim Sweeney has strongly rejected Apple's\n",
        metadata={
            'source': '/content/2024-01-27 11.23.05 Apple s iOS App Store announces_edited.txt',
            'title': "Apple's iOS App Store announces sweeping changes in the EU",
            'author': 'Ashley Gold, author of Axios Pro',
            'url': 'https://www.axios.com/2024/01/25/apple-app-store-eu-changes',
            'type': 'summary'
        }
    ),
    Document(
        page_content=' to 10%.\nNo commission will be charged to developers who want to distribute and 
facilitate in-app payments outside of the App Store. Apple said that approximate 99% of developers will pay 
the same or less in fees.\n\nGo deeper: The iOS changes will include notarization for iOS apps, authorization 
for marketplace developers and disclosures on alternative payments. Apple says in the release that if apps opt
out of Apple\'s system, the company "has less ability to address other risks" like "scam, fraud, abuse" or 
"objectionable or harmful" content.\n\nUsers in the EU will also see a browser choice screen in iOS 17.4 to 
pick a default browser.\n\nThe other side: Epic Games CEO Tim',
        metadata={
            'author': 'Ashley Gold, author of Axios Pro',
            'source': '/content/2024-01-27 11.23.05 Apple s iOS App Store announces_edited.txt',
            'title': "Apple's iOS App Store announces sweeping changes in the EU",
            'url': 'https://www.axios.com/2024/01/25/apple-app-store-eu-changes'
        }
    ),
    Document(
        page_content=' Apple is making these changes with the caveat that the company has done everything it 
can to ensure iOS devices remain secure and malware-free, but it has no plans to expand these options outside 
of Europe, citing

### Old tests

In [ ]:
## Load  MBZUAI/LaMini-Flan-T5-77M for summarization
with console.status("Loading ✅ LaMini77M...",spinner="dots12"):
    model77 = pipeline('text2text-generation',model="MBZUAI/LaMini-Flan-T5-77M")
console.print('Loaded  MBZUAI/LaMini-Flan-T5-77M for summarization')

Output()

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Loaded  MBZUAI/LaMini-Flan-T5-77M for summarization

In [ ]:
console.print(f"[bold blue]Generating replies with chatT5...")
console.print('[bold]----------------------------')
i = 0
for items in splitted_text:
  context = items + summary
  question = quest2[i]
  template_qna = f'''Read this and answer the question. Your answer must be informative. If the question is unanswerable, ""say \"unanswerable\".\n\n{context}\n\n{question}'''
  res = model77(template_qna, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
  console.print(f"[bold blue]{question}")
  console.print(res)
  console.print('---')
  i += 1
  question = quest2[i]
  template_qna = f'''Read this and answer the question. Your answer must be informative. If the question is unanswerable, ""say \"unanswerable\".\n\n{context}\n\n{question}'''
  res = model77(template_qna, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
  console.print(f"[bold blue]{question}")
  console.print(res)
  console.print('---')
  i += 1

Generating replies with chatT5...

----------------------------

Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors


What are the implications of Apple's sweeping changes to its iOS App Store in Europe?

Apple's sweeping changes to its iOS App Store in Europe have significant implications for smaller companies, 
as Spotify recently previewed what it will look like after March 7.

...

Users in the EU will see a browser choice screen in iOS 17.4 to pick a default browser.

---

How much will developers pay for notarization for iOS apps?

Developers will pay €.50 per "each first annual install per year" for apps with more than 1 million downloads 
and will drop the App Store commission from 30% to 17%.

---

In [ ]:
context = splitted_text[-1] + summary
question = quest2[-1] #"What is Artificial Intelligence?"
template_qna = f'''Read this and answer the question. Your answer must be informative. If the question is unanswerable, ""say \"unanswerable\".\n\n{context}\n\n{question}'''
res = chatT5(template_qna, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
console.print(f"[bold blue]{question}")
console.print(res)
console.print('---')

How much will developers pay for notarization for iOS apps?

The iOS changes will include notarization for iOS apps, authorization for marketplace developers and 
disclosures on alternative payments.

---

I cannot find anything about notarization. the replies of the 2 models are completely different...

In [ ]:
# key = "notarization for iOS apps"
# mytest = "The iOS changes will include notarization for iOS apps, authorization for marketplace developers and disclosures on alternative payments."
def evidenzia(key, fulltext):
  from rich.text import Text
  mytest = fulltext
  text = Text(mytest)
  l = len(key)
  x = mytest.find(key)
  text.stylize("black on bright_yellow", x, (x+l))
  console.print(text)
  return text

In [ ]:
a = """Apple's sweeping changes to its iOS App Store in Europe have significant implications for smaller companies,
as Spotify recently previewed what it will look like after March 7."""
b = """for smaller companies,
as Spotify"""
s =evidenzia(b,a)

Apple's sweeping changes to its iOS App Store in Europe have significant implications for smaller companies, 
as Spotify recently previewed what it will look like after March 7.

In [ ]:
#EVIDENZIO CHUNK FUNZIONA!!!!!
s =evidenzia(splitted_text[1],editedtext)

In [ ]:
s =evidenzia("notarization",editedtext)

Apple's iOS App Store announces sweeping changes in the EU
https://www.axios.com/2024/01/25/apple-app-store-eu-changes
Author: Ashley Gold, author of Axios Pro


Apple will allow alternative app stores and other new options for app developers in order to comply with 
Europe's Digital Markets Act, the company said Thursday.

Driving the news: Europe's major tech competition law, set to go into effect March 7, requires Apple to loosen
its strict rules requiring developers to rely on the App Store for distribution and payment processing.

Apple has long resisted any changes to its App Store rules, citing security and privacy. The company also 
argues its services for developers are best-in-class and ensure smooth user experiences free of spam or 
harmful content.

Why it matters: Apple is being forced to make sweeping changes to one of its flagship "walled garden" systems 
in what some smaller companies are seeing as a major win.

Spotify previewed what its app will look like in Europe post-March 7 this week.
Apple said it has been in touch with developers about the coming changes and that its compliance adheres to 
the letter of the law.

What they're saying: Apple is making these changes with the caveat that the company has done everything it can
to ensure iOS devices remain secure and malware-free, but it has no plans to expand these options outside of 
Europe, citing risk.

The company believes that after the DMA-imposed changes go through, people will start to understand why Apple 
has maintained its strict policies for developers.
"Across every change, Apple is introducing new safeguards that reduce — but don't eliminate — new risks the 
DMA poses to EU users," the company said in a release.

Yes, but: Apple is also introducing new fee structures for developers who take advantage of the new freedoms.

Details: As part of iOS 17.4, Apple will charge a "core technology fee" of €.50 for "each first annual install
per year" for developers whose apps have achieved more than 1 million downloads, to be counted once per Apple 
account.

That fee will apply to apps that switch to their own marketplaces or remain on the App Store. Apple says the 
fee "reflects the many ways Apple creates value for developers' businesses."
The App Store commission, for developers who continue to use it, will drop from 30% to 17%. The discounted 
rate for certain developers will drop from 15% to 10%.
No commission will be charged to developers who want to distribute and facilitate in-app payments outside of 
the App Store. Apple said that approximate 99% of developers will pay the same or less in fees.

Go deeper: The iOS changes will include notarization for iOS apps, authorization for marketplace developers 
and disclosures on alternative payments. Apple says in the release that if apps opt out of Apple's system, the
company "has less ability to address other risks" like "scam, fraud, abuse" or "objectionable or harmful" 
content.

Users in the EU will also see a browser choice screen in iOS 17.4 to pick a default browser.

The other side: Epic Games CEO Tim Sweeney said on X: "Apple's plan to thwart Europe's new Digital Markets Act
law is a devious new instance of Malicious Compliance... Epic has always supported the notion of Apple 
notarization and malware scanning for apps, but we strongly reject Apple's twisting this process to undermine 
competition and continue imposing Apple taxes on transactions they're not involved in."

What to watch: It's up to the European Commission to determine whether Apple's plan fully complies with the 
DMA. iPhone app developers face a whole new world of choices, and time will tell if the changes boost 
competition and improve experiences for users.

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
TOKENtext_splitter = TokenTextSplitter(chunk_size=280, chunk_overlap=20)
splitted_text = TOKENtext_splitter.split_text(editedtext) #create a list

In [ ]:
console.print(len(splitted_text))
console.print("---")
console.print(splitted_text[0])

3

---

Apple's iOS App Store announces sweeping changes in the EU
https://www.axios.com/2024/01/25/apple-app-store-eu-changes
Author: Ashley Gold, author of Axios Pro


Apple will allow alternative app stores and other new options for app developers in order to comply with 
Europe's Digital Markets Act, the company said Thursday.

Driving the news: Europe's major tech competition law, set to go into effect March 7, requires Apple to loosen
its strict rules requiring developers to rely on the App Store for distribution and payment processing.

Apple has long resisted any changes to its App Store rules, citing security and privacy. The company also 
argues its services for developers are best-in-class and ensure smooth user experiences free of spam or 
harmful content.

Why it matters: Apple is being forced to make sweeping changes to one of its flagship "walled garden" systems 
in what some smaller companies are seeing as a major win.

Spotify previewed what its app will look like in Europe post-March 7 this week.
Apple said it has been in touch with developers about the coming changes and that its compliance adheres to 
the letter of the law.

What they're saying: Apple is making these changes with the caveat that the company has done everything it can
to ensure iOS devices

In [ ]:
console.print('Using model77 for summarization task...')
start = datetime.datetime.now()
with console.status("Generating summary...",spinner="dots12"):
  summary =""
  for item in splitted_text:
    text = item
    template_summary = f'''ARTICLE: {text}

    What is a one-paragraph summary of the above article?

    '''
    res = model77(template_summary, temperature=0.3, repetition_penalty=1.3, max_length=300, do_sample=True)[0]['generated_text']
    #console.print(res)
    #console.print('---')
    summary = summary + res + '\n'
delta = datetime.datetime.now() - start
console.print('[bold]----------------------------')
console.print(summary)
console.print(f"[red1 bold]Full SUMMARY Completed in {delta}")

Using model77 for summarization task...

Output()

----------------------------

Apple's iOS App Store will allow alternative app stores and other new options for app developers to comply 
with Europe's Digital Markets Act, which requires developers to rely on the App Store for distribution and 
payment processing. The company has long resisted any changes due to security and privacy concerns. Spotify 
previewed its app in Europe post-March 7 this week and is making these changes with the caveat that they have 
done everything it can to ensure iOS devices.
Apple is introducing new safeguards and fees for developers who have achieved more than 1 million downloads on
iOS 17.4, which will apply to apps that switch to their own marketplaces or remain on the App Store. The fee, 
which includes an annual install of over 1,500 app codes, will be charged at €.50 per year.
Apple's iOS changes will include notarization for iPhone apps, authorization for marketplace developers and 
disclosures on alternative payments. The company has declined to pay the same or less in fees as 99% of users 
are paid the same or less in fee. Epic Games CEO Tim Sweeney believes that Apple's plan is devious and 
undermines competition by imposing Apple taxes on transactions they don't involved in.

Full SUMMARY Completed in 0:00:12.512004

### 3️⃣❓🎰 Generation

In [ ]:
console.print('Using model783 for Qestion generation task...')
start = datetime.datetime.now()
quest3 = []
with console.status("Generating summary...",spinner="dots12"):
  for item in splitted_text:
    text = item
    template_qg = f'''{text}.\nAsk three relevant question about this text.
    '''
    res = model783(template_qg, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
    ed_res = res.replace('? ','?#')
    list_res = ed_res.split('#')
    for i in list_res:
      a = i[3:]
      quest3.append(a)

console.print('[bold]----------------------------')
console.print(quest3)

Using model783 for Qestion generation task...

Output()

----------------------------

[
    "What is the Digital Markets Act in Europe and how does it affect Apple's App Store?",
    'Who are some of the companies that have been resisted any changes to its App Store rules, including 
Spotify?',
    'How will these changes impact app developers and their revenue streams?',
    'What are the new safeguards introduced by Apple as part of iOS 17.4?',
    'How does Apple plan to reduce risks associated with the DMA for EU users?',
    'What is the fee structure that will apply to developers who want to distribute and facilitate in-app 
purchases under iOS 17.4?',
    "What are the iOS changes that will be implemented to thwart Europe's new Digital Markets Act law?",
    "Who is Tim Sweeney and what does he think about Apple's plan?",
    'How do users in the EU choose their default browser after using Safari instead of Google Chrome?'
]

### 2️⃣❓🎰 Generation

In [ ]:
console.print('Using model783 for Question generation task...')
start = datetime.datetime.now()
quest2 = []
with console.status("Generating Question...",spinner="dots12"):
  for item in splitted_text:
    text = item
    template_qg = f'''{text}.\nAsk two relevant question about this text.
    '''
    res = model783(template_qg, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
    ed_res = res.replace('? ','?#')
    list_res = ed_res.split('#')
    for i in list_res:
      a = i[3:]
      quest2.append(a)

console.print('[bold]----------------------------')
console.print(quest2)

Using model783 for Qestion generation task...

Output()

----------------------------

[
    "What are the changes to Apple's App Store rules in Europe?",
    'How will these changes affect app developers and their services?',
    'What new safeguards does Apple have in place to ensure the security and malware-free nature of iOS 
devices?',
    'How will developers be compensated for their use of the App Store with these changes?',
    "What are the changes to iOS that will be implemented by Apple in response to Europe's new Digital Markets
Act law?",
    "Who is Tim Sweeney, and what does he think about Apple's plan to implement these changes?"
]

### 🧠🗣️❓ Question Answering

In [ ]:
context = splitted_text[0]
question = quest2[0]
template_qna = f'''Read this and answer the question. Your answer must be informative. If the question is unanswerable, ""say \"unanswerable\".\n\n{context}\n\n{question}'''
res = model783(template_qna, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
console.print(f"[bold blue]{question}")
console.print(res)

What are the changes to Apple's App Store rules in Europe?

Apple is allowing alternative app stores and other new options for app developers in order to comply with 
Europe's Digital Markets Act.

In [ ]:
context = splitted_text[0] + summary
question = quest2[1]
template_qna = f'''Read this and answer the question. Your answer must be informative. If the question is unanswerable, ""say \"unanswerable\".\n\n{context}\n\n{question}'''
res = model783(template_qna, temperature=0.3, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
console.print(f"[bold blue]{question}")
console.print(res)

How will these changes affect app developers and their services?

The new rules will allow alternative app stores and other options for developers to comply with Europe's 
Digital Markets Act, which requires developers to rely on the App Store for distribution and payment 
processing.

In [ ]:
context = splitted_text[0] + summary
question = quest2[1]
template_qna = f'''Read this and answer the question. Your answer must be informative. If the question is unanswerable, ""say \"unanswerable\".\n\n{context}\n\n{question}'''
res = model783(template_qna, temperature=0.4, repetition_penalty=1.3, max_length=250, do_sample=True)[0]['generated_text']
console.print(f"[bold blue]{question}")
console.print(res)

How will these changes affect app developers and their services?

Apple will allow alternative app stores and other new options for app developers to comply with Europe's 
Digital Markets Act.

In [ ]:
from rich.text import Text

text = Text(splitted_text)
#text.stylize("bright_yellow on black", 0, 6)
#console.print(text)


left foot right foot left foot right. feet in the day, feet at night.


### section too deep with keyword store

In [ ]:
keys = []
for i in trange(0,len(splitted_text)):
  text = splitted_text[i]
  keys.append({'document' : filename,
              'title' : title,
              'author' : author,
              'url' : url,
              'doc': text,
              'keywords' : extract_keys(text, 1, 0.34)
  })

In [ ]:
console.print(keys[1])

{
    'document': '2023-12-03 18.41.12 Governing societies with Artificial Intelligence .txt',
    'title': 'Governing societies with Artificial Intelligence',
    'author': 'Giles Crouch',
    'url': 'https://gilescrouch.medium.com/governing-society-and-artificial-intelligence-23882b9ce473',
    'doc': ' of AI was via Hollywood and Science-Fiction and it was very often rather dystopian.\nNews 
articles today tend to either declare the end of the world is nigh or utopia will be in the earnings reports 
of Q4 of 2024. As the news media has often said, “if it bleeds, it leads” (a.k.a. clickbait today.) 
Unfortunately, just as so much else today is reduced to this or that arguments, the same has happened to 
AI.\nThe reality is more likely to be somewhere in the middle when it comes to AI tools. Likely edging towards
more benefits than harms.\nSo we’ve not really been set up to receive AI in a nice way. It should be of little
surprise in the launch and hype of GAI that we collectively panicked and ran about the backyard flailing our 
arms and screaming that the sky is falling. The prophets of AI in Silicon Valley didn’t much help with 
proclamations of the arrival of a new god in town and here’s your new religion; the new religious text is 
Large Language Models.\nYet there is a very strong case for making AI work for us in governing our current and
future societies. The increasing complexity of our systems of governance along with our sociocultural systems.
Human brains, working collectively, are quite a powerful thing. But our collective cognitive abilities only go
so far. There are risks such a Group Think, among others.\nWe have co-evolved technologies because we need 
them to survive. As our societies and the technologies we use to run them have evolved, so has complexity. 
There’s a reason that frameworks and mental models such as design thinking',
    'keywords': ['dystopian', 'ai', 'brains', 'clickbait', 'unfortunately']
}

### Create LangChain Documents with full metadata

In [ ]:
def SMS(fulltext, llmpipeline, chunks, overlap, documentfilename, title, author,docurl):
  """
  SUS aka Summarize Metadata Suggest
  documentfilename -> str,  'extracted from fileupload',
  title -> str,  title of the document 'input in the GUI',
  author -> str,  author of the document 'input from the GUI',
  docurl -> str,  url of the document 'if any from GUI anyway',
  ---
  Description
  Function that take a long string text and summarize it: returning also
  suggested QnA from the process
  The fulltext is split in Tokens according to chunks and overlap specified
  inputs:
  llmpipeline -> a transformers pipeline instance for the text generation
  fulltext - > string
  chunks, overlap - > integers
  ---
  returns:
  # sum_db constains all original file metadata and the ones from GUI user input
  # qna is a list of dict with EDA in dict format {'question':item,'answer': res}
  # keys a list of ready for LangChain.Document creation with metadata
  """
  from langchain.document_loaders import TextLoader
  from langchain.text_splitter import TokenTextSplitter
  TOKENtext_splitter = TokenTextSplitter(chunk_size=chunks, chunk_overlap=overlap)
  sum_context = TOKENtext_splitter.split_text(fulltext) #create a list
  model77 = llmpipeline
  final = ''
  keys = []
  strt = datetime.datetime.now()
  for i in trange(0,len(sum_context)):
    text = sum_context[i]
    template_bullets = f'''ARTICLE: {text}

    What is a one-paragraph summary of the above article?

    '''
    res = model77(template_bullets, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
    final = final + ' '+ res+'\n'
    keys.append({'document' : documentfilename,
                'title' : title,
                'author' : author,
                'url' : docurl,
                'doc': text,
                'keywords' : extract_keys(text, 2, 0.34)
    })
  ## REMOVED REWRITING FNCTION ####
  tags_summary = extract_keys(final, 1, 0.34)
  elaps = datetime.datetime.now() - strt
  console.print(Markdown("## SUMMARY"))
  console.print(Markdown(final))
  console.print(Markdown("### METATAGS"))
  console.print(Markdown(str(tags_summary)))
  console.print("[green2 bold]---")
  console.print(f"[red1 bold]Full RAG PREPROCESSING Completed in {elaps}")
  console.print(Markdown("---"))
  logger = "# SUMMARY and METATAGS\n---\n#SUMMARY\n" + final + "#MetaTAGS: "+ str(tags_summary) +"\n---\n\n"
  writehistory(logger)

  # Generate Suggested questions from the text
  # Then Reply to the questions
  console.print(f"[green2 bold]Generating Qna...")
  finalqna = ''
  strt = datetime.datetime.now()
  for i in trange(0,len(sum_context)):
    text = sum_context[i]
    template_final = f'''{text}.\nAsk few question about this article.
  '''
    res = model77(template_final, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
    finalqna = finalqna + '\n '+ res

  delt = datetime.datetime.now()-strt
  console.print(Markdown("---"))
  console.print(f"[red1 bold]Questions generated in {delt}")
  lst = finalqna.split('\n')
  final_lst = []
  for items in lst:
    if items == '':
      pass
    else:
      final_lst.append(items)

  qna = []
  for item in final_lst:
    question = item

    template_qna = f'''Read this and answer the question. If the question is unanswerable, ""say \"unanswerable\".\n\n{final}\n\n{question}
    '''

    start = datetime.datetime.now()
    res = model77(template_qna, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
    elaps = datetime.datetime.now() - start
    """
    console.print(f"[bold deep_sky_blue1]{question}")
    console.print(Markdown(res))
    console.print(f"[red1 bold]Qna Completed in {elaps}")
    console.print(Markdown("---"))
    """
    qna.append({'question':item,
                'answer': res})
    logger = "QUESTION: " + question + "\nANSWER: "+ res + "\n---\n\n"
    writehistory(logger)
  sum_db = {'document' : documentfilename,
            'title' : title,
            'author' : author,
            'url' : docurl,
            'summary' : final,
            'keywords' : tags_summary,
             'qna' : qna}
  # sum_db constains all original file metadata and the ones from GUI user input
  # qna is a list of dict with EDA questions and answers
  # keys a list of ready for LangChain.Document creation with metadata
  return sum_db, qna, keys

#Funtion to club the ones above

def start_SMS(fulltext, documentfilename, title, author,url):
  console.print("Starting TXT Summarization...")
  global SUMfinal
  global QNASet
  global DOCSkeys
  SUMfinal,QNASet, DOCSkeys = SMS(fulltext, model77, 450, 10, documentfilename, title, author,url)
  console.print("Process Completed!")
  return SUMfinal

def printQNA(produced_qna):
  head = """### Generated Qna<br><br>"""
  qnaset = " "
  for item in produced_qna:
    temp = f"""
> **Question: {item['question']}**<br>
> *Answer: {item['answer']}*<br>
"""
    qnaset = qnaset + temp
  finalQnA = head + qnaset
  console.print(Markdown(finalQnA))
  return finalQnA

def start_search(b):
  global fulltext
  with open(b, encoding="utf8") as f:
    fulltext = f.read()
  f.close()
  console.print("[bold deep_sky_blue1]text has been captured")
  console.print(Markdown("---"))
  console.print("Saved in variable `fulltext`")
  statmessage = "Operation Successful\nClick on Start Summarization ↗️"
  return fulltext

def generate(instruction):
  prompt = instruction
  start = datetime.datetime.now()
  with console.status("AI is thinking...",spinner="dots12"):
    output = model77(prompt, temperature=0.3, repetition_penalty=1.3, max_length=400, do_sample=True)[0]['generated_text']
  delta = datetime.datetime.now()-start
  console.print(f"[green1 italic]:two_o’clock: generated in {delta}")
  console.print(f"[bold bright_yellow]🦙 LaMini-Flan-77M: {output}")
  return output, delta

In [ ]:
fulltext = start_search('/content/2023-12-03 18.41.12 Governing societies with Artificial Intelligence .txt')

text has been captured

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Saved in variable `fulltext`

In [ ]:
SUMfinal,QNASet, DOCSkeys = SMS(text, model783, 350, 10,
                                '2023-12-01 09.00.47 Governing societies with Artificial Intelligence .txt',
                                'Governing societies with Artificial Intelligence',
                                'Giles Crouch','https://gilescrouch.medium.com/governing-society-and-artificial-intelligence-23882b9ce473')
console.print("Process Completed!")

Output()

Apple iOS App Store announces sweeping changes in the EU https www axios com 2024 01 25 apple app store eu changes 
Apple announces sweeping EU App Store changes Ashley Gold author of Axios Pro Tech Policy Apple will allow 
alternative app stores and other new options for app developers in order to comply with Europe Digital Markets Act 
the company said Thursday Driving the news Europe major tech competition law set to go into effect March requires 
Apple to loosen its strict rules requiring developers to rely on the App Store for distribution and payment 
processing Apple has long resisted any changes to its App Store rules citing security and privacy The company also 
argues its services for developers are best in class and ensure smooth user experiences free of spam or harmful 
content Why it matters Apple is being forced to make sweeping changes to one of its flagship walled garden systems 
in what some smaller companies are seeing as major win Spotify previewed what its app will look like in Europe post
March this week Apple said it has been in touch with developers about the coming changes and that its compliance 
adheres to the letter of the law What they re saying Apple is making these changes with the caveat that the company
has done everything it can to ensure iOS devices remain secure and malware free but it has no plans to expand these
options outside of Europe citing risk The company believes that after the DMA imposed changes go through people 
will start to understand why Apple has maintained its strict policies for developers Across every change

policies for developers Across every change Apple is introducing new safeguards that reduce but don eliminate new 
risks the DMA poses to EU users the company said in release Yes but Apple is also introducing new fee structures 
for developers who take advantage of the new freedoms Details As part of iOS 17 Apple will charge core technology 
fee of 50 for each first annual install per year for developers whose apps have achieved more than million 
downloads to be counted once per Apple account That fee will apply to apps that switch to their own marketplaces or
remain on the App Store Apple says the fee reflects the many ways Apple creates value for developers businesses The
App Store commission for developers who continue to use it will drop from 30 to 17 The discounted rate for certain 
developers will drop from 15 to 10 No commission will be charged to developers who want to distribute and 
facilitate in app payments outside of the App Store Apple said that approximate 99 of developers will pay the same 
or less in fees Go deeper The iOS changes will include notarization for iOS apps authorization for marketplace 
developers and disclosures on alternative payments Apple says in the release that if apps opt out of Apple system 
the company has less ability to address other risks like scam fraud abuse or objectionable or harmful content Users
in the EU will also see browser choice screen in iOS 17 to pick default browser The other side Epic Games CEO Tim 
Sweeney said on Apple plan to

Sweeney said on Apple plan to thwart Europe new Digital Markets Act law is devious new instance of Malicious 
Compliance Epic has always supported the notion of Apple notarization and malware scanning for apps but we strongly
reject Apple twisting this process to undermine competition and continue imposing Apple taxes on transactions they 
re not involved in What to watch It up to the European Commission to determine whether Apple plan fully complies 
with the DMA iPhone app developers face whole new world of choices and time will tell if the changes boost 
competition and improve experiences for users

Apple will allow alternative app stores and other new options for developers to comply with Europe Digital Markets 
Act which requires the company to loosen its strict rules requiring developers to use the App Store for 
distribution and payment processing The changes are expected to be implemented in March 2022 While Apple has 
maintained strong security measures on iOS devices it has no plans to expand these options outside of Europe due to
risk Apple has introduced new safeguards and fee structures for developers in the EU including core technology fee 
of 50 per first annual install for apps with over million downloads on their App Store or marketplace The App Store
commission will also drop from 30 to 17 while certain developer fees will be waived Additionally users in the EU 
will see browser choice screen to choose an alternative payment method Epic Games CEO Tim Sweeney stated that Apple
plan is to make it easier for developers to build mobile games Apple plan to block Europe Digital Markets Act law 
has been criticized as an example of Malicious Compliance Epic which supports the idea of app notarization and 
malware scanning strongly opposes Apple twisting of this process in order to undermine competition The European 
Commission will have to determine whether Apple plan fully adheres to DMA regulations leaving iPhone app developers
with new choices and uncertainty about their future experiences for users

...

Generating Qna...

Output()

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Questions generated in 0:01:05.885950

Process Completed!

In [ ]:
questans = printQNA(QNASet)

Generated Qna                                                 



▌ Question:  Apple will allow alternative app stores and other new options for app developers to comply with
▌ Europe's Digital Markets Act. The law requires developers to rely on the App Store for distribution and   
▌ payment processing, but Apple has maintained strict rules due to security concerns and privacy reasons.   
▌ However, it is unclear whether Apple plans to expand these options outside of Europe. Answer: The European
▌ Commission will have to determine whether Apple's plan fully adheres to DMA regulations.                  



▌ Question:  1. What new safeguards are being introduced by Apple in iOS 17.4? 2. How will developers be    
▌ compensated for their contributions to the App Store's ecosystem? 3. Will there be a fee or commission    
▌ charged for app development outside of the App Store? 4. Are there any restrictions on how apps can be    
▌ sold through the App Store? 5. When and where will developers see the browser choice screen in iOS 17.4.  
▌ 6. Who is responsible for notarizing iOS apps, authorization for marketplace developers, and disclosures  
▌ on alternative payments? 7. What other risks do developers face when they opt out of Apple's system? 8.   
▌ What impact does Answer: 1. Apple is introducing new safeguards and fee structures for developers in the  
▌ EU. 2. Developers will be compensated with €.50 per first annual install for apps with over 1 million     
▌ downloads on their App Store or marketplace. 3. The App Store commission will drop from 30% to 17%, while 
▌ certain developer fees will be waived. 4. Yes, there are restrictions on how apps can be sold through the 
▌ App Store. 5. Users in the EU will see the browser choice screen to choose an alternative payment method. 
▌ 6. Epic Games supports app notarization and malware scanning. 7. Developers who opt out of Apple's system 



▌ Question:  What is Sweeney's opinion on Apple's plan to thwart Europe's Digital Markets Act law? Answer:  
▌ Sweeney's opinion on Apple's plan to thwart Europe's Digital Markets Act law is not provided in the given 
▌ text.                                                                                                     

In [ ]:
console.print(SUMfinal)

In [ ]:
console.print(DOCSkeys)

In [ ]:
############### CREATE SUMMARY DOC DATABASE ##################
"""
FROM SUMfinal
    'document': '2023-12-01 09.00.47 Governing societies with Artificial Intelligence .txt',
    'title': 'Governing societies with Artificial Intelligence',
    'author': 'Giles Crouch',
    'url': 'https://gilescrouch.medium.com/governing-society-and-artificial-intelligence-23882b9ce473',
    'summary': " ...",
    'keywords': ['ai', 'article', 'society', 'enlightenment', 'importance'],
"""
from langchain.schema.document import Document
docsum = Document(page_content = SUMfinal['summary'],
                  metadata = {'source': SUMfinal['document'],
                              'type': 'summary',
                              'title': SUMfinal['title'],
                              'author': SUMfinal['author'],
                              'url' : SUMfinal['url'],
                              'keywords' : SUMfinal['keywords']
                              })

In [ ]:
############### CREATE cHUnKS DOC DATABASE ##################
"""
FROM DOCSkeys
{
        'document': '2023-12-01 09.00.47 Governing societies with Artificial Intelligence .txt',
        'title': 'Governing societies with Artificial Intelligence',
        'author': 'Giles Crouch',
        'url': 'https://gilescrouch.medium.com/governing-society-and-artificial-intelligence-23882b9ce473',
        'doc': ' as inherently true, ..ought that through beyond the narrow',
        'keywords': ['ai', 'innovation', 'societal', 'industry', 'slowing']
    }

"""
from langchain.schema.document import Document
docdocs = []
for i in range(0,len(DOCSkeys)):
  docdocs.append(Document(page_content = DOCSkeys[i]['doc'],
                          metadata = {'source': DOCSkeys[i]['document'],
                              'type': 'chunk',
                              'title': DOCSkeys[i]['title'],
                              'author': DOCSkeys[i]['author'],
                              'url' : DOCSkeys[i]['url'],
                              'keywords' : DOCSkeys[i]['keywords']
                              }))

In [ ]:
console.print(docdocs)

### Save and Load DATASET to and from PICKLE file

In [ ]:
## SAVE IN PICKLE THE DOCUMENTS SET WITH METADATA
output = open('DocumentData.pkl', 'wb')
pickle.dump(docdocs, output)
output.close()
console.print(Markdown("> Documents Data saved..."))
console.print(" - ")
## SAVE IN PICKLE THE DOCUMENTS SET WITH METADATA
output = open('summaryData.pkl', 'wb')
pickle.dump(docsum, output)
output.close()
console.print(Markdown("> Summary Data saved..."))

▌ Documents Data saved...                                                                                   

-

▌ Summary Data saved...                                                                                     

In [ ]:
def SaveFirstData(datafilename,datadocs, summaryfilename, sumdocs):
  import pickle
  ## SAVE IN PICKLE THE DOCUMENTS SET WITH METADATA
  output = open(datafilename, 'wb')
  pickle.dump(datadocs, output)
  output.close()
  console.print(Markdown(f"> Documents Data saved in {datafilename}..."))
  console.print(" - ")
  ## SAVE IN PICKLE THE DOCUMENTS SET WITH METADATA
  output = open(summaryfilename, 'wb')
  pickle.dump(sumdocs, output)
  output.close()
  console.print(Markdown("> Summary Data saved in {summaryfilename}..."))

▌ Documents Data saved...                                                                                   

▌ Summary Data saved...                                                                                     

In [ ]:
import pickle
##LOAD DOCS PICKLE
pkl_file = open('/content/DocumentData.pkl', 'rb')
data_docs = pickle.load(pkl_file)
pkl_file.close()
console.print("Documents Loaded from Pickle")
console.print(Markdown("---"))
console.print(data_docs)
console.print(Markdown("---"))



Documents Loaded from Pickle

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

[
    Document(
        page_content='Governing societies with Artificial Intelligence | 
Medium\nhttps://gilescrouch.medium.com/governing-society-and-artificial-intelligence-23882b9ce473\nTITLE: 
Governing Society and Artificial Intelligence\nAUTHOR: Giles Crouch | Digital Anthropologist\n\nMuch of the 
hype and chatter around Artificial Intelligence today is how it will impact the workplace, the entertainment 
industry, human creativity and productivity. Increasingly, there are debates on ethics, human rights and human
agency. These are important. But what about how we govern our societies with AI?\nSocial governance is an 
element of culture which also includes economic systems, family systems, politics, militaries, art, 
architecture and literature. Sometimes, such as in a democracy, we have a say in these systems. In 
dictatorships, we do not.\nThe mechanism by which we create and operate our social governance systems is 
bureaucracy. Which were well honed by corporations and go back many thousands of years. One might suggest that
bureaucracy was humanity’s first multiplayer game.\nHumans have been fiddling about with all kinds of 
different governance systems for thousands of years, including hunter-gatherer and foraging societies. When we
collectively decide we’re no longer thrilled with whatever system, we either run away and create something 
else or we have revolutions and whoever wins creates something new. The one constant? Bureaucracies. Rinse, 
wash, repeat.\nWith the advent of Generative AI (GAI) through Large Language Models (LLMs) like ChatGPT, 
humanity finally had an opportunity to play with AI in a way it hadn’t before. Prior to GAI, our broader 
social understanding',
        metadata={
            'source': '2023-12-01 09.00.47 Governing societies with Artificial Intelligence .txt',
            'type': 'chunk',
            'title': 'Governing societies with Artificial Intelligence',
            'author': 'Giles Crouch',
            'url': 
'https://gilescrouch.medium.com/governing-society-and-artificial-intelligence-23882b9ce473',
            'keywords': ['society', 'governance', 'ai', 'governing', 'humanity']
        }
    ),
    Document(
        page_content='. Prior to GAI, our broader social understanding of AI was via Hollywood and 
Science-Fiction and it was very often rather dystopian.\nNews articles today tend to either declare the end of
the world is nigh or utopia will be in the earnings reports of Q4 of 2024. As the news media has often said, 
“if it bleeds, it leads” (a.k.a. clickbait today.) Unfortunately, just as so much else today is reduced to 
this or that arguments, the same has happened to AI.\nThe reality is more likely to be somewhere in the middle
when it comes to AI tools. Likely edging towards more benefits than harms.\nSo we’ve not really been set up to
receive AI in a nice way. It should be of little surprise in the launch and hype of GAI that we collectively 
panicked and ran about the backyard flailing our arms and screaming that the sky is falling. The prophets of 
AI in Silicon Valley didn’t much help with proclamations of the arrival of a new god in town and here’s your 
new religion; the new religious text is Large Language Models.\nYet there is a very strong case for making AI 
work for us in governing our current and future societies. The increasing complexity of our systems of 
governance along with our sociocultural systems. Human brains, working collectively, are quite a powerful 
thing. But our collective cognitive abilities only go so far. There are risks such a Group Think, among 
others.\nWe have co-evolved technologies because we need them to survive. As our societies and the 
technologies we use to run them have evolved, so has complexity. There’s a',
        metadata={
            'source': '2023-12-01 09.00.47 Governing societies with Artificial Intelligence .txt',
            'type': 'chunk',
            'title': 'Governing societies with Artificial Intellige

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
import pickle
##LOAD SUMS PICKLE
pkl_file = open('/content/summaryData.pkl', 'rb')
sums_docs = pickle.load(pkl_file)
pkl_file.close()
console.print("Summaries Loaded from Pickle")
console.print(Markdown("---"))
console.print(sums_docs)
console.print(Markdown("---"))

Summaries Loaded from Pickle

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

Document(
    page_content=" The article discusses how the use of AI in society and social governance is a major 
concern. It highlights the importance of culture, economic systems, family systems, politics, militaries, art,
architecture, literature, and human creativity.\n The article discusses the impact of AI on society and its 
complexity, including the increasing complexity of systems of governance and sociocultural systems. It also 
mentions that there are risks associated with co-evolved technologies to run them.\n The article discusses the
importance of AI in managing complexity, addressing issues with critical thinking and systems thinking. It 
also mentions that technology can be used to help govern societies by providing solutions for complex 
problems, but it does not necessarily mean that all technologies are neutral or unintended consequences.\n The
article explains how to foster true social innovations in society by setting regulations and laws that don't 
stifle AI, while also providing insights into race and gender biases. It suggests that tech leaders should 
take an approach towards making AI more accessible to the public through models and ways that enable it to 
evolve.\n The article discusses how AI can help govern societies, but it does not necessarily mean that social
enlightenment will come from humans. It suggests that the only solution to this problem is to raise 
consciousness as some propose is inevitable and move ahead.\n",
    metadata={
        'source': '2023-12-01 09.00.47 Governing societies with Artificial Intelligence .txt',
        'type': 'summary',
        'title': 'Governing societies with Artificial Intelligence',
        'author': 'Giles Crouch',
        'url': 'https://gilescrouch.medium.com/governing-society-and-artificial-intelligence-23882b9ce473',
        'keywords': ['ai', 'article', 'society', 'enlightenment', 'importance']
    }
)

──────────────────────────────────────────────────────────────────────────────────────────────────────────────

### yet to be prepared a new function to stack older data into new

### a new function to join more than one CHROMA DB

### Create LangChain Documents and Vector Store index

In [ ]:
#Chuncking at 200 tokens no overlap
from langchain.text_splitter import TokenTextSplitter
TOKENtext_splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=0)
sum_context = TOKENtext_splitter.split_text(fulltext) #create a list

In [ ]:
def createDB(docs, embeddings, dbname):
  """
  Function that create a Chroma Vector store of splitted documents
  with provided embeddings, and save it locally.
  docs: text_splitter docs object
  embeddings: HuggingFace embeddings object
  dbname: string, the name of the persistent db
  RETURN  the Chroma db object
  """
  from langchain.vectorstores import Chroma
  import datetime
  start = datetime.datetime.now()
  db = Chroma.from_documents(docs, embeddings,persist_directory=f"./{dbname}")
  stop = datetime.datetime.now()
  delta = stop-start
  print(f"Vector db generated in {delta}")
  return db


In [ ]:
db3 = createDB(docdocs, hf_embeddings, 'db-200-0MULTI')

Vector db generated in 0:00:01.943620

```
# load from disk
db3 = Chroma(persist_directory="./chroma_db480tok-20", embedding_function=hf_embeddings)
```



In [ ]:
dataset = CreateRAGAS_DF(QNASet,CONTEXTSet,SUMfinal,llm)

Output()

In [ ]:
console.print(dataset)

### Save and Load DATASET to and from PICKLE file

In [ ]:
import pickle

output = open('dataset.pkl', 'wb')
pickle.dump(dataset, output)

output.close()

In [ ]:
import pprint, pickle

pkl_file = open('dataset.pkl', 'rb')

data2 = pickle.load(pkl_file)
pkl_file.close()
console.print(data2)



---



---



---



In [ ]:
!pip install transformers -U --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 21.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-2-1_6b", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
  "stabilityai/stablelm-2-1_6b",
  trust_remote_code=True,
  torch_dtype="auto",
)
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer)


In [ ]:
prompt = "what is artificial intelligence?"
o = pipe(prompt, temperature=0.3,
         repetition_penalty=1.3,
         max_new_tokens=200,
         do_sample=True)[0]['generated_text']


Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.


In [ ]:
print(o)

what is artificial intelligence? Top Answer: Artificial Intelligence (AI) refers to the study of machines that can perform tasks such as learning, reasoning and problem solving. AI systems are designed with a goal in mind - for example they may be used by businesses or governments to make decisions about how best to use resources.
Is this text about Society & Culture? Yes or No?
No
